# Chart Analysis Functions

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from spotifyData import spotifyData
from chartArtistAlbumData import chartArtistAlbumData
from chartUtils import *
from extraArtists import extraKnownArtists


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-02-03 20:17:54.272003


# Rename Input Data

In [69]:
updateMultiDB(2)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  Initializing a fresh DB for multi
  There are currently 0 artist keys.
  There are currently 0 renamed artist keys.
  Forced reload of 136 artist names
Saving 136 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 5.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  --> This file is 5.8kB.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 136 artist keys.
  There are currently 142 renamed artist keys.


In [70]:
manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)

def updateMultiDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 2:
        tmpmanDB = masterArtistNameDB("multi", init=True)
        tmpmanDB.forceReload(getFile("multi.yaml"))
        tmpmanDB.save()
        
        multimanDB = masterArtistNameDB("multi", init=False)
        saveFile(idata=multimanDB.getRenames(), ifile="multi.yaml")

def updateManDB(step=1):
    if step == 1:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.checkForRecursives()

    if step == 2:
        tmpmanDB = masterArtistNameDB("main", init=True)
        tmpmanDB.forceReload(getFile("main.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    if step == 3:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.checkForRecursives()
        
    if step == 4:
        tmpmanDB = masterArtistNameDB("main", init=False)
        tmpmanDB.addRenames(getFile("relDBRenames2.yaml"))
        tmpmanDB.save()
        
        manDB = masterArtistNameDB("main", init=False)
        saveFile(idata=manDB.getRenames(), ifile="main.yaml")
        
    
    _, _ = clock("Last Ran")

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7149 artist keys.
  There are currently 8333 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 136 artist keys.
  There are currently 142 renamed artist keys.


In [ ]:
updateMultiDB(2)

# Reverse Chart Lookup

In [ ]:
updateMultiDB(1)

In [ ]:
updateMultiDB(2)

In [ ]:
updateManDB(1)

In [ ]:
updateManDB(2)

In [ ]:
from multiArtist import multiartist
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
mularts.setKnownMultiDelimArtists(getFile("../multiartist/knownMultiArtists.yaml"))
knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
print(len(knownMultiArtists),"Known Artists")
        
ignoresComp = ['Soundtrack', 'Various Artists', 'Original Broadway Cast Recording', 'Read-Along','Veggietales', 'Pokemon', 
               'Barney', 'Walt Disney Read-Along', 'The Powerpuff Girls',
               'Superstar Kidz', 'Original Cast', 'Original Cast Recording']
ignores = ignoresComp    
    
def matchArtist(artistName):
    subNames = mularts.getArtistNames(artistName)
    result = {subName: mdbmap.isKnownByName(subName) for subName in subNames}
    for subName in result.keys():
        if subName in ignores:
            result[subName] = True
    return result


for chartName in chartDFs.keys():
    try:
        chartDFs[chartName]['DB'] = chartDFs[chartName]['RenamedArtist'].apply(matchArtist)
    except:
        print("Error with {0}".format(chartName))

In [ ]:
vals = {}
for chartName in chartDFs.keys():
    try:
        vals[chartName] = chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))].shape[0]
    except:
        pass
from pandas import Series

In [ ]:
Series(vals).sort_values(ascending=False)

In [ ]:
mdbmap.isKnownByName("Salt-N-Pepa")

In [ ]:
chartDF = chartDFs["vinyl-albums"]
def getMissingArtists(chartDF):
    missing = chartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]
    return list(set(getFlatList(missing["DB"].apply(lambda x: [k for k,v in x.items() if v is False]))))
getMissingArtists(chartDF)

In [ ]:
def getMissingArtists(chartDF):
    ichartDF[~chartDF["DB"].apply(lambda x: all(x.values()))]

In [ ]:
chartName = "independent-albums"
chartDFs[chartName][~chartDFs[chartName]["DB"].apply(lambda x: all(x.values()))]

In [ ]:
bMap = {}
for chartTitle,chartName in initMap.items():
    newName = newMap.get(chartTitle)
    if newName is not None:
        bMap[newName] = chartName


In [ ]:
saveFile(ifile="billboardMapping.p", idata=bMap)

In [ ]:
bMap = getFile("billboardMapping.p")

# Top40 Data

In [ ]:
%load_ext autoreload
%autoreload
from top40Data import top40Data
td = top40Data(minYear=1, maxYear=2021)
td.setChartUsage(rank=[0,1,2,3])
td.setDBRenames(manDB)
td.setMultiDBRenames(multimanDB)
td.setFullChartData()
td.setArtistAlbumData()
td.saveArtistAlbumData()
td.saveFullChartData()

_, _ = clock("Last Run")

# Billboard Data

In [ ]:
%load_ext autoreload
%autoreload
from billboardData import billboardData
bd = billboardData(minYear=1, maxYear=2021)
bd.setChartUsage(rank=[0,1,2,3,4])
bd.setDBRenames(manDB)
bd.setMultiDBRenames(multimanDB)
bd.setFullChartData()
bd.setArtistAlbumData()

bd.saveArtistAlbumData()
bd.saveFullChartData()

_, _ = clock("Last Run")

# BillboardYE Data

In [46]:
%load_ext autoreload
%autoreload
from billboardYE import billboardYE
bYE = billboardYE(minYear=1, maxYear=2021)
bYE.setChartUsage(rank=[0,1,2,3,4,5,6])
bYE.setDBRenames(manDB)
bYE.setFullChartData()
bYE.setArtistAlbumData()

bYE.saveFullChartData()
bYE.saveArtistAlbumData()

_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  Getting Chart For hot
  Using 5 Charts
  Using 5 Charts
  Getting Chart For adult
  Using 4 Charts
  Using 4 Charts
  Getting Chart For alternative
  Using 7 Charts
  Getting Chart For countryMusic
  Using 15 Charts
  Getting Chart For rock
  Using 17 Charts
  Getting Chart For rnb
  Using 37 Charts
  Using 53 Charts
  Getting Chart For alternative
  Using 7 Charts
  Getting Chart For countryMusic
  Using 15 Charts
  Getting Chart For rock
  Using 17 Charts
  Getting Chart For rnb
  Using 37 Charts
  Using 76 Charts
  Getting Chart For christian
  Using 32 Charts
  Getting Chart For canadian
  Using 4 Charts
  Getting Chart For comedy
  Using 2 Charts
  Getting Chart For radio
  Using 2 Charts
  Using 30 Charts
  Getting Chart For christian
  Using 32 Charts
  Getting Chart For canadian
  Using 4 Charts
  Getting Chart For comedy
  Using 2 Charts
  Getting Chart For radio
  Using 2 Charts
  Using 

Renamed 3159 single artists
Saving 3055 Full Artist Data
Saving data to currentBillboardYEFullChartArtistAlbumData.p
  --> This file is 309.4kB.
Saved data to currentBillboardYEFullChartArtistAlbumData.p
  --> This file is 309.4kB.
Saving 3055 Artist Album Data to currentBillboardYEArtistAlbumData.p
Saving data to currentBillboardYEArtistAlbumData.p
  --> This file is 150.8kB.
Saved data to currentBillboardYEArtistAlbumData.p
  --> This file is 150.8kB.
Current Time is Wed Feb 03, 2021 20:52:35 for Last Run


# Spotify Code

In [76]:
%load_ext autoreload
%autoreload
from spotifyData import spotifyData
sd = spotifyData(minYear=1, maxYear=2021)
sd.setDBRenames(manDB)
sd.setMultiDBRenames(multimanDB)
sd.setChartUsage(rank=[8])
sd.setFullChartData()
sd.setArtistAlbumData()
sd.saveArtistAlbumData()
sd.saveFullChartData()
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
  Getting Chart For brazil
name = brazil
  Using 1 Charts
  Using 1 Charts
  Using Charts (None): ['regional-br-weekly']
Found 9 summary files
  Using regional-br-weekly
Renamed 599 single artists
Renamed 8 multi artists
Saving 501 Artist Album Data to currentSpotifyArtistAlbumData.p
Saving data to currentSpotifyArtistAlbumData.p
  --> This file is 33.1kB.
Saved data to currentSpotifyArtistAlbumData.p
  --> This file is 33.1kB.
Saving 501 Full Artist Data
Saving data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 50.6kB.
Saved data to currentSpotifyFullChartArtistAlbumData.p
  --> This file is 50.6kB.
Current Time is Wed Feb 03, 2021 21:24:21 for Last Run


# Common Code

In [ ]:
updateMultiDB(2)

# Load Chart Analysis

In [7]:
#mcm    = matchChartMusic(mdb)
recreate = True
if recreate:
    maindb = mainDB(mdb=None, create=True, debug=True)
    maindb.setDBArtists(recreate=False)
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()
else:
    maindb = mainDB(mdb=None, create=False, debug=False)
    maindb.loadDBDataMap()    
_, _ = clock("Last Run")

Setting Basic Database Objects
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rateyourmusic   =========================
=========================   deezer   =========================
Database Records:
  Creating Database Records for Discogs
  Creating Database Records for AllMusic
  Creating Database Records for MusicBrainz
  Creating Database Records for LastFM
  Creating Database Records for RateYourMusic
  Creating Database Records for Deezer
Available DBs: Discogs, AllMusic, MusicBrainz, LastFM, RateYourMusic, Deezer
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7148 artist keys.
  There are currently 8332 rena

Summary Statistics For DB: MusicBrainz
    Using Known Artists: False
    Found 204225 ID -> Name entries
    Found 172906 Name -> ID entries
    Found 0 Albums
=========================   lastfm   =========================
Summary Statistics For DB: LastFM
    Using Known Artists: False
    Found 11005 ID -> Name entries
    Found 10917 Name -> ID entries
    Found 137222 Albums
=========================   rateyourmusic   =========================
Summary Statistics For DB: RateYourMusic
    Using Known Artists: False
    Found 657 ID -> Name entries
    Found 655 Name -> ID entries
    Found 0 Albums
=========================   deezer   =========================
Summary Statistics For DB: Deezer
    Using Known Artists: False
    Found 5922 ID -> Name entries
    Found 5919 Name -> ID entries
    Found 0 Albums
Current Time is Wed Feb 03, 2021 20:34:07 for Last Run


# Analyze Data

In [77]:
## Basic stuff
%load_ext autoreload
%autoreload
from chartArtistAlbumData import chartArtistAlbumData


singleArtistAlbumData = {}
manyArtistAlbumData   = {}

chartType = "Spotify"
mType     = "Full"
cad = chartArtistAlbumData(chartType, ignoreMultiNames=False)
cad.createIndivArtistAlbumData()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
chartArtistAlbumData(Spotify)
Found 62 files.
Adding 504 known multi delim artists.
  Assigning 504 known multi-name artists
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7152 artist keys.
  There are currently 8338 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 136 artist keys.
  There are currently 142 renamed artist keys.
There are 501 unique artists in artist albums data
There are 501 newly unique artists in artist albums data
Renamed 0 artists
Renamed 0 artists (multi)
There are 501    unique artist entries in the artist albums data
There are 468    single artist entries in the artist albums data
There ar

In [ ]:
mdbmap = musicDBMap("Spotify")

In [ ]:
[{'Vince Guaraldi Trio': 7471},{'Eliot Morris': 1821351},{'Ross Lynch': 3150221},
 {'The Cheetah Girls': 15341}, {'Pinkfong': 12919403}, {'Hannah Montana': 16805},
 {'Big Time Rush': 877741}, {'Cedarmont Kids': 1399450}, {'Kidz Bop Kids', 320265},
 {'The Countdown Kids': 1685087}]

# Create Master Match Class

In [78]:
def showMatchedStatus(chartType, mdbmaps, mdbmc):
    print("Matched {0} / {1} Artists for {2} ChartType".format(mdbmaps[chartType].getSize(), len(mdbmc.matchData[chartType]), chartType))
    
def showRemainingMatches(chartType, mdbmc):
    print('='*100)
    print("="*10,"Artists That Were Not Tested (Low Albums)",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)
    print("="*10,"Previously Tested Artists Without A Match",'='*10)
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    print('='*100)    
    
def analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=0.1):
    dbRenames = {}
    toget = {}
    for primaryKey,artistName in mdbmaps[chartType].getArtists().items():
        primaryArtistName = artistName
        artistData = mdbmaps[chartType].getArtistDataByKey(primaryKey)
        for db,dbID in artistData.getDict().items():
            if db in ["DatPiff", "MetalStorm", "LastFM", "CDandLP", "AceBootlegs", "RockCorner", "MusicStack", "RateYourMusic"]:
                continue
            if dbID is not None:
                secondaryArtistName = maindb.getArtistDBNameFromID(db, dbID)
                if secondaryArtistName is None:
                    if toget.get(db) is None:
                        toget[db] = []
                    toget[db].append(primaryArtistName)
                    continue
                s = SequenceMatcher(None, primaryArtistName, secondaryArtistName)
                ratio = s.ratio()
                maxRatio = maxVal
                if ratio >= maxRatio:
                    continue

                if ratio >= maxRatio-diffVal:
                    if dbRenames.get(secondaryArtistName) is not None:
                        if dbRenames[secondaryArtistName] == primaryArtistName:
                            continue
                        else:
                            pass
                    dbRenames[secondaryArtistName] = primaryArtistName
                    print("[{0: <30} {1: <4} {2: >30}] \t --> ({3}) {4} / {5} ({6})".format(primaryArtistName,round(ratio,2),secondaryArtistName,chartType,primaryKey,dbID,db))


    print("Found {0} artists to get.".format(len(toget)))
    print("Found {0} artists to rename.".format(len(dbRenames)))
    return toget, dbRenames    
    
def removeDBRenames(mdbmaps, chartType, dbRenames):
    for k,v in dbRenames.items():
        kKey = mdbmaps[chartType].getPrimaryKey(artistName=k, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)

        kKey = mdbmaps[chartType].getPrimaryKey(artistName=v, artistID=None)
        if kKey is not None:
            mdbmaps[chartType].removeArtistByKey(kKey)
    mdbmaps[chartType].save()
    
    
def analyzeRenames(manDB, dbRenames):
    redos = {}
    dels = []
    if len(dbRenames) == 0:
        print("Nothing to process.")
        return
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys() and bestname not in manDB.artistNameDB.keys():
            print(oldername,bestname)

            redos[bestname] = oldername
            dels.append(oldername)
            print('\t',oldername,'\t',bestname)

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))
    for oldername in dels:
        del dbRenames[oldername]

    print("# Redos: {0}".format(len(redos)))
    dbRenames.update(redos)
    print("# Renames: {0}".format(len(dbRenames)))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))
    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")




    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    def isAscii(ele):
        return len(ele) == len(ele.encode())

    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if isAscii(v) and not isAscii(k):
            if '’' in k or "“" in k:
                continue
            redos[v] = k
            dels.append(k)
            print('\t',k,'\t',v)
    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")








    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if " the " in v and " The " in k:
            if k == v.replace(" the ", " The "):
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")







    dbRenames = getFile("relDBRenames2.yaml")
    redos = {}
    dels = []
    for k,v in dbRenames.items():    
        if '’' in v or "“" in v:
            if not '’' in k and not "“" in v:
                redos[v] = k
                dels.append(k)
                print('\t',k,'\t',v)

    #print(len(dbRenames))
    for k in dels:
        del dbRenames[k]

    #print(len(dbRenames))
    dbRenames.update(redos)
    #print(len(dbRenames))


    saveFile(idata=dbRenames, ifile="relDBRenames2.yaml")
    #print(len(dbRenames))
    
    
        
    
def manualAppends(toMatch, chartType, minAlbums=0, add=True):
    if len(toMatch[chartType]) == 0:
        print("Nothing to append...")
        return
    
    toMatchDF = DataFrame({item[0]: {"ArtistName": item[1]["ArtistName"], "ArtistAlbums": item[1]["ArtistAlbums"]} for item in toMatch[chartType]}).T
    toMatchDF["NumAlbums"] = toMatchDF["ArtistAlbums"].apply(len)
    toMatchDF = toMatchDF.sort_values(by="NumAlbums", ascending=False)
    
    for idx,row in toMatchDF.iterrows():
        if add:
            artistName = row["ArtistName"]
            albumNames = row["ArtistAlbums"]
            nAlbums    = row["NumAlbums"]
            
            if nAlbums < minAlbums:
                continue
            print("add{0}(mdbmaps, {1}, {2}, {3})   ### [{4}]".format("AllMusic", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            print("add{0}(mdbmaps, {1}, {2}, {3})    ### [{4}]".format("Deezer", "\"{0}\"".format(chartType), "\"{0}\"".format(idx), "\"{0}\"".format(""), artistName))
            print("### {0}".format("  ,  ".join(albumNames)))
            for albumName in albumNames:
                print("### {0}  ---> [{1}]".format(albumName, cad.getAlbumsData(albumName)))
            print("\n")
        else:
            print("ignores.append(\"{0}\")".format(artistName))
            if " & " in artistName:
                continue
                print("ignores.append(\"{0}\")".format(artistName))

    print("mdbmaps[{0}].save()".format("\"{0}\"".format(chartType)))    
    





        
def copyMapData(mdbmap):
    return
    mdbmap.saveCopy()

def saveMapData(mdbmap, result_list):
    for item in result_list:
        primaryKey = item[0] 
        artistName = item[1]
        artistID   = item[2]
        mcs        = item[3]
        mdbmap.addArtistByKey(primaryKey, artistName=artistName, artistID=artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistDataByKey(primaryKey, db, matchID)

    mdbmap.save()
    

def matchItAll(mdbmaps, thresholds, useAlbums=True, mdbmc=None, db=None, toMatch=None):
    if toMatch is None:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=10000, maxAlbums=50000)

    if len(toMatch) == 0:
        return

    num_processes = 3
    if useAlbums is True:
        func = matchDBArtistWithAlbums
    else:
        func = matchDBArtistWithoutAlbums
    pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
    #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
    dbName = list(toMatch.keys())[0]
    copyMapData(mdbmaps[dbName])

    argument_list = toMatch[dbName]
    if len(argument_list) == 0:
        return
    print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
    result_list = multiProc(func=pfunc, argument_list=argument_list,
                                           num_processes=num_processes)


    start, cmt = clock("Saving...")
    saveMapData(mdbmaps[dbName], result_list)
    elapsed(start, cmt)
    print("\nSleeping for 10 seconds...\n")
    #sleep(10)
    
    #mdbmc.matchMutualMaps()

    
def matchDBArtistWithoutAlbums(item, *args, **kwargs):
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = None
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score']),
    mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    
    primaryKey = item[0]
    artistData = item[1]
    artistName   = artistData["ArtistName"]
    artistID     = primaryKey
    artistAlbums = artistData["ArtistAlbums"]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                             matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                             matchScore=kwargs['score'])
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [primaryKey,artistName,artistID,mcs]
    return retval


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm

In [79]:
mdbmaps = {}
print("ChartType = {0}".format(chartType))
print("MType     = {0}".format(mType))
mdbmaps[chartType] = musicDBMap(chartType, init=True, copy=False)
mdbmc = masterDBMatchClass(maindb, mdbmaps)

manDB      = masterArtistNameDB("main", init=False)
multimanDB = masterArtistNameDB("multi", init=False)


def reMatch(mdbmaps=mdbmaps, mdbmc=mdbmc, mtype="Single"):
    if mtype == "Single":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getSingleArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Many":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getManyArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
    elif mtype == "Full":
        matchData = {mdbmaps[chartType].getPrimaryKey(artistName=artistName, artistID=None): {"ArtistName": artistName, "ArtistAlbums": artistAlbums} for artistName,artistAlbums in cad.getFullArtistAlbumData().items()}
        mdbmc.setDBMatchData(chartType, matchData)
        
reMatch(mtype=mType)

ChartType = Spotify
MType     = Full
  Loaded 0 previously matched entries
Loading Artist Names
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7152 artist keys.
  There are currently 8338 renamed artist keys.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/multiArtistNameDB.p
  There are currently 136 artist keys.
  There are currently 142 renamed artist keys.
  Setting matchData for Spotify


## Match Using Names And Albums

In [80]:
ignoresList = ["chartIgnores.yaml"]
ignoresList = []

ignoresList = []
ignoresList.append("Artists Stand Up To Cancer")
ignoresList.append("Cast Of Rent")
ignoresList.append("2006 Broadway Cast Recording")
ignoresList.append("Original Broadway Cast Of Dear Evan Hansen")
ignoresList.append("Various")
ignoresList.append("Various Artists")
ignoresList.append("Read-Along")
ignoresList.append("Varios Artistas")
ignoresList.append("Artists Against Bullying")
ignoresList.append("Varios")
ignoresList.append("Sleep Music Lullabies")
ignoresList.append("Yoga Meditation Tribe")
ignoresList.append("The Cast Of Stomp")
ignoresList.append("Quincy Jones Feat. The Cast Of Stomp/The Yes/No Productions")
ignoresList.append("Walt Disney Read-Along")
ignoresList.append("Classical Lullabies For Babies Academy")
ignoresList.append("Diverse")
saveFile(idata=ignoresList, ifile="chartIgnoreArtists.yaml")

unknownList = []
unknownList.append("Amil-lion")
unknownList.append("Yung Lito")
unknownList.append("Idols")
unknownList.append("X Factor Finalists")
unknownList.append("R")
unknownList.append("Malena")
unknownList.append("Neales")
saveFile(idata=unknownList, ifile="chartUnknownArtists.yaml")

internationalList = []
internationalList.append("3robiu")
internationalList.append("Multi-interpretes")
internationalList.append("Mormon Tabernacle Choir/Orch. At Temple Square/Yoncheva/Mumford/Villazon/Terfel (Wilberg)")
internationalList.append("Villazon")
internationalList.append("Mumford")
internationalList.append("Vienna Philharmonic (Dudamel)")
internationalList.append("Louisville Orchestra (Abrams)")
internationalList += ['Eight Ranger', 'Yi Yang Qian Xi', "ONE N' ONLY", "MoLa", "Sus Invitados"]
internationalList += ['Juanma Y Su Tuna Para Todo El Ano', 'NYC', 'Nogizaka 48', 'Sho Kiryuin from Golden Bomber']
internationalList += ['The MONSTERS', 'ZERO-G', 'Huang Lige', 'Renzo', 'Bodan Shonen Dan', 'ST RISH', 'King Cream Soda']
internationalList += ['Good Morning America', 'Hotta Ke BAND', 'Feng Jianyu', 'Stanley Most']
internationalList += ['Cardiff City Fc', 'Leeds United Team', 'Monkey Hangerz']
internationalList += ['Good Child Foundation', 'Neon Brotherhood', 'Rogue Souljahz']
internationalList += ['GMC', 'Krossfade', 'Dave Howley', 'Toko', 'Y-not', 'Afro Music']
internationalList += ['Husvenne', 'Maca Boy', 'Skyndeep', 'TwitterXmassingle', 'Julieanne Dineen', 'Julie-Anne Dineen', 'Anybodies']
saveFile(idata=internationalList, ifile="chartInternationalArtists.yaml")

tonewUnknownList  = ['Lil Adrei', 'The 7 Dimensions', 'HIXTAPE', 'Porky Slim']
tonewUnknownList += ['SL', 'Sidemen', 'Mastermind', 'HOSH', 'Frosty']
saveFile(idata=tonewUnknownList, ifile="chartToNewArtists.yaml")

{'Gabo Parisi':9846490}
{'Rory & The Island': 1219224, 'Sean Og': 537158, "Hoo Leeger": 10908080}
{'The Suspects and Guests': 5061058, "Dennis Reed & Gap": 15080705}
{}

{"Eri Esittajia": 243610}

soundtrackList = ['Jul På Vesterbro']
saveFile(idata=soundtrackList, ifile="chartSoundtrackArtists.yaml")

applemusicList  = ['Dutch','Deno','DHT','Sadbh']
applemusicList += ['DDG', 'Lean Trap', 'MOLO', 'Amina']
applemusicList += ['Rocket Girls 101', "David Miles", "Cyril"]
applemusicList += ["Camur"]
saveFile(idata=applemusicList, ifile="chartAppleMusicArtists.yaml")

ignoresList = ["chartIgnoreArtists.yaml", "chartUnknownArtists.yaml", "chartInternationalArtists.yaml",
               "chartToNewArtists.yaml", "chartAppleMusicArtists.yaml", "chartSoundtrackArtists.yaml"]


In [81]:
# THOMAS
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
reMatch(mtype=mType)
print("Done.\n\n")

========== 50
Total                -> 527
After Ignores        -> 527
After Known          -> 527
After MaxAlbums      -> 527
After MinAlbums      -> 1
After MaxValues      -> 1
Running imap multiprocessing for 1 artists ...


100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Current Time is Wed Feb 03, 2021 21:25:11 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 472B.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 472B.
Current Time is Wed Feb 03, 2021 21:25:11 for Done with Saving...
Process [Done with Saving...] took 6.0 millseconds.

Sleeping for 10 seconds...

========== 20
Total                -> 527
After Ignores        -> 527
After Known          -> 526
After MaxAlbums      -> 526
After MinAlbums      -> 18
After MaxValues      -> 18
Running imap multiprocessing for 18 artists ...



100%|██████████| 18/18 [00:04<00:00,  4.41it/s]

Current Time is Wed Feb 03, 2021 21:25:15 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 3.7kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 3.7kB.
Current Time is Wed Feb 03, 2021 21:25:15 for Done with Saving...
Process [Done with Saving...] took 14.0 millseconds.

Sleeping for 10 seconds...

========== 10
Total                -> 527
After Ignores        -> 527
After Known          -> 508
After MaxAlbums      -> 508
After MinAlbums      -> 29
After MaxValues      -> 29
Running imap multiprocessing for 29 artists ...


...

Current Time is Wed Feb 03, 2021 21:25:58 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 33.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 33.8kB.
Current Time is Wed Feb 03, 2021 21:25:58 for Done with Saving...
Process [Done with Saving...] took 95.0 millseconds.

Sleeping for 10 seconds...

========== 2
Total                -> 527
After Ignores        -> 527
After Known          -> 344
After MaxAlbums      -> 344
After MinAlbums      -> 79
After MaxValues      -> 79
Running imap multiprocessing for 79 artists ...


100%|██████████| 79/79 [00:23<00:00,  3.35it/s]


Current Time is Wed Feb 03, 2021 21:26:22 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 46.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 46.8kB.
Current Time is Wed Feb 03, 2021 21:26:22 for Done with Saving...
Process [Done with Saving...] took 132.0 millseconds.

Sleeping for 10 seconds...

========== 1
Total                -> 527
After Ignores        -> 527
After Known          -> 265
After MaxAlbums      -> 265
After MinAlbums      -> 265
After MaxValues      -> 265
Running imap multiprocessing for 265 artists ...


100%|██████████| 265/265 [01:33<00:00,  2.82it/s]


Current Time is Wed Feb 03, 2021 21:27:56 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 93.5kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 93.5kB.
Current Time is Wed Feb 03, 2021 21:27:57 for Done with Saving...
Process [Done with Saving...] took 281.0 millseconds.

Sleeping for 10 seconds...

  Setting matchData for Spotify
Done.




In [82]:
showMatchedStatus(chartType, mdbmaps, mdbmc)

Matched 527 / 527 Artists for Spotify ChartType


## Match Only Using Names

#### Show The Remaining Artists

In [83]:
showRemainingMatches(chartType, mdbmc)

========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========
Total                -> 527
After Ignores        -> 527
After DB Matches     -> 357
After MaxAlbums      -> 357
After MinAlbums      -> 357
After MaxValues      -> 357


#### Try To Match Artists That Were Previously Tested Without A Match

In [84]:
for i in range(1):
    toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
    if len(toMatch[chartType]) == 0:
        break
    matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

Total                -> 527
After Ignores        -> 527
After DB Matches     -> 357
After MaxAlbums      -> 357
After MinAlbums      -> 357
After MaxValues      -> 357
Running imap multiprocessing for 357 artists ...


100%|██████████| 357/357 [01:28<00:00,  4.02it/s]


Current Time is Wed Feb 03, 2021 21:29:27 for Saving...
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 100.8kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 100.8kB.
Current Time is Wed Feb 03, 2021 21:29:27 for Done with Saving...
Process [Done with Saving...] took 276.0 millseconds.

Sleeping for 10 seconds...

  Setting matchData for Spotify


In [85]:
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Matched 527 / 527 Artists for Spotify ChartType
========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========
Total                -> 527
After Ignores        -> 527
After DB Matches     -> 217
After MaxAlbums      -> 217
After MinAlbums      -> 217
After MaxValues      -> 217


#### Try To Match Artists That Were Not Previously Tested (Low Albums)

In [86]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=True, ignores=ignoresList)

In [87]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(1.0), toMatch=toMatch, useAlbums=False)
reMatch(mtype=mType)

  Setting matchData for Spotify


In [88]:
showRemainingMatches(chartType, mdbmc)

========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========
Total                -> 527
After Ignores        -> 527
After DB Matches     -> 217
After MaxAlbums      -> 217
After MinAlbums      -> 217
After MaxValues      -> 217


#### Extra Artists Info

In [89]:
from extraArtists import extraKnownArtists
extraKnownArtists(mdbmaps, chartType)
showMatchedStatus(chartType, mdbmaps, mdbmc)
showRemainingMatches(chartType, mdbmc)

Adding known artist data for Spotify
Could not add artist data from PrimaryKey: cb93fcbd3ab132ac5f5e531955b18314
Could not add artist data from PrimaryKey: 7431b4cd1d5849d221baaa7888604a2e
Could not add artist data from PrimaryKey: 30662e7c962665273622d2ccd451c3f6
Could not add artist data from PrimaryKey: a04480bfdbafbc14b45846754ba8740e
Could not add artist data from PrimaryKey: 404f6b7df28cbd293554f353dc5976b2
Could not add artist data from PrimaryKey: dd9d8d514ab3ca6bc2ec2c9480cd67bf
Could not add artist data from PrimaryKey: 1909ad8603f5b973026a364c6d6cee8e
Could not add artist data from PrimaryKey: c595fb8cc61d6b512b253fc4ad505b32
Could not add artist data from PrimaryKey: ef9f608a3737c743051a6a27cc74d8e4
Could not add artist data from PrimaryKey: f215decc2db538a85c1cdd3159a21fba
Could not add artist data from PrimaryKey: c83e0d312315ea129a3e5039add19073
Could not add artist data from PrimaryKey: b6fbc16cce329059ad1fe561da88d6a3
Could not add artist data from PrimaryKey: d29ca9cd

  --> This file is 100.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicdb/dbSpotifyMap.p
  --> This file is 100.9kB.
Matched 527 / 527 Artists for Spotify ChartType
========== Artists That Were Not Tested (Low Albums) ==========
========== Previously Tested Artists Without A Match ==========
Total                -> 527
After Ignores        -> 527
After DB Matches     -> 217
After MaxAlbums      -> 217
After MinAlbums      -> 217
After MaxValues      -> 217


#### Unmatched Artists

In [90]:
toMatch   = mdbmc.getDataToMatch(chartType, maxValues=10000, maxAlbums=100000, minAlbums=0, useKnown=False, dbMatches=0, ignores=ignoresList)
len(toMatch[chartType])

Total                -> 527
After Ignores        -> 527
After DB Matches     -> 217
After MaxAlbums      -> 217
After MinAlbums      -> 217
After MaxValues      -> 217


217

In [ ]:
toget

In [ ]:
cad.getAlbumsData("Stories")

In [ ]:
#cad.getAlbumsData("The Very Best Of")
updateMultiDB(2)

In [ ]:
toget

In [ ]:
updateManDB(2)

In [ ]:
[item[1]["ArtistName"] for item in toMatch["Spotify"]]

In [91]:
manualAppends(toMatch, chartType, minAlbums=0, add=True)

addAllMusic(mdbmaps, "Spotify", "48ba1711710a9403e914ee24ebe47ccc", "")   ### [Cristiano]
addDeezer(mdbmaps, "Spotify", "48ba1711710a9403e914ee24ebe47ccc", "")    ### [Cristiano]
### Seu Polícia - Ao Vivo  ,  Cadeira de Aço - Ao Vivo  ,  Amigo Taxista  ,  Largado As Traças - Acústico  ,  Bebida Na Ferida - Acústico  ,  A Gente Continua - Acústico  ,  Status Que Eu Não Queria - Acústico  ,  Novela das Nove - Acústico  ,  Largado Às Traças - Ao Vivo  ,  Status Que Eu Não Queria - Ao Vivo  ,  Bebida Na Ferida - Ao Vivo  ,  A Gente Continua - Ao Vivo  ,  Novela das Nove - Ao Vivo  ,  Notificação Preferida - Ao Vivo  ,  Derreter a Aliança - Ao Vivo  ,  Armadura - Ao Vivo  ,  Mulher Maravilha - Ao Vivo  ,  Morador de Rua - Ao Vivo  ,  A Farra Perdeu Pro Amor - Ao Vivo  ,  Grão de Arroz - Ao Vivo  ,  Amor À Primeira Esquina - Ao Vivo  ,  Ontem Era Eu - Ao Vivo  ,  Estado Decadente - Acústico  ,  Long Neck - Acústico  ,  Whisky e Gelo - Acústico  ,  Sobrando Ausência - Acústico  ,  Enchendo e 

In [ ]:
addAllMusic(mdbmaps, "Top40", "6a3b1f200312ef0462e7cc93ab2c386e", "mn0000263577")   ### [Feiern Im Regen]
addDiscogs(mdbmaps, "Top40", "6a3b1f200312ef0462e7cc93ab2c386e", "255362")    ### [Feiern Im Regen]
### Die Toten Hosen
### Die Toten Hosen  ---> [['Feiern Im Regen']]


addAllMusic(mdbmaps, "Top40", "ad613f10cc09bcc4f1347439ebc38ce7", "mn0000655705")   ### [Dj Otzi]
addDiscogs(mdbmaps, "Top40", "ad613f10cc09bcc4f1347439ebc38ce7", "381181")    ### [Dj Otzi]
### Tranen
### Tranen  ---> [['Dj Otzi & Kate Hall']]



addAllMusic(mdbmaps, "Top40", "1d307f027d49d11331cc03ece90e4a00", "mn0000390743")   ### [Ok Go]
### Here It Goes Again  ,  Get Over It
### Here It Goes Again  ---> [['Ok Go']]
### Get Over It  ---> [['Ok Go', 'Guillemots']]


addDiscogs(mdbmaps, "Top40", "0d2d0e133d441072a6065b1400890ee1", "277352")    ### [Searchers]
### The Very Best Of
### The Very Best Of  ---> [['Sheryl Crow', 'Michael Bolton', 'Aerosmith', 'Fleetwood Mac', 'Eagles', 'Prince', 'The Stone Roses', 'The Jackson 5', 'Lighthouse Family', 'The Doors', 'Sting & The Police', 'Dolly Parton', 'Cat Stevens', 'Bay City Rollers', 'The Jacksons', 'Roy Orbison', 'Johnny Mathis', 'Nina Simone', 'The Platters', 'Searchers', 'Bobby Vee', 'Jim Reeves']]


addDiscogs(mdbmaps, "Top40", "249b4aad4574f7718a083398f0a6d07e", "1229202")    ### [Lmfao]
### Let The Bass Kick In Miami Bitch
### Let The Bass Kick In Miami Bitch  ---> [['Chuckie & Lmfao']]



addAllMusic(mdbmaps, "Top40", "49294a3c9a6f375dd00565815a66effc", "mn0000071514")   ### [Bob Marley]
### Live Forever  ,  Legend: The Best Of Bob Marley And The Wailers  ,  Legend: The Best Of Bob Marley
### Live Forever  ---> [['Bob Marley']]
### Legend: The Best Of Bob Marley And The Wailers  ---> [['Bob Marley']]
### Legend: The Best Of Bob Marley  ---> [['Bob Marley']]


addAllMusic(mdbmaps, "Top40", "3f378fbb8c2be2109efad066f4902d58", "mn0002690553")   ### [The Neighbourhood]
### I Love You  ,  Wiped Out!
### I Love You  ---> [['The Neighbourhood']]
### Wiped Out!  ---> [['The Neighbourhood']]


mdbmaps["Top40"].save()

In [ ]:
addDiscogs(mdbmaps, "Top40", "977d879a9ef599db0179842a90eb0829", "2986568")    ### [Nik & Ras]
### Fugt I Fundamentet  ,  Hvad Der Sker Her
### Fugt I Fundamentet  ---> [['Nik & Ras']]
### Hvad Der Sker Her  ---> [['Nik & Ras']]

addAllMusic(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "mn0001560738")   ### [Milk & Honey]
addDiscogs(mdbmaps, "Top40", "27b4e2613c000e26afc0dcbf6ab8137a", "506822")    ### [Milk & Honey]
### Didi
### Didi  ---> [['Milk & Honey']]

addDiscogs(mdbmaps, "Top40", "4fb5ad8251f32ebc71a4caa19a79671c", "1404551")    ### [Rald Schmitz]
### Schaun Und Das Schaf
### Schaun Und Das Schaf  ---> [['Rald Schmitz']]

addDiscogs(mdbmaps, "Top40", "35a5e92b985eb2530625c792eb8794fd", "2261153")    ### [Ell & Nikki]
### Running Scared
### Running Scared  ---> [['Ell & Nikki']]

addDiscogs(mdbmaps, "Top40", "108e43f1076de906cdd8ba014d4b16ff", "2068922")    ### [Bonez Mc]
### Gottseidank
### Gottseidank  ---> [['Trettmann, Bonez Mc & RAF Camora']]

addDiscogs(mdbmaps, "Top40", "225ac6dbdf0fffd4a9e113da51ca214d", "6081758")    ### [Brudi30]
### Ghetto
### Ghetto  ---> [['Samra, Capital Bra, Brudi30 & Kalazh44']]

addDiscogs(mdbmaps, "Top40", "446d29ccb5a2a3de8ac79412defdc3b0", "4255795")    ### [YouNotUs]
### Narcotic
### Narcotic  ---> [['YouNotUs, Janieck & Senex']]

addDiscogs(mdbmaps, "Top40", "c3516450970181d37943c2cd92289bfd", "4367398")    ### [Anthony]
### Do Ya
### Do Ya  ---> [['Anthony & Jasmin']]

addDiscogs(mdbmaps, "Top40", "df31d4c938e84e9804579b48368955ba", "4453381")    ### [Eicka Jane]
### Hojre Og Venstre
### Hojre Og Venstre  ---> [['Skinz & Eicka Jane', 'Skinz & Ericka Jane']]

addDiscogs(mdbmaps, "Top40", "5921f9df84764cd6c69a4f92a205f390", "7671579")    ### [Lolo]
### Pub G
### Pub G  ---> [['Lolo, Branco & Larry 44']]


mdbmaps["Top40"].save()

In [ ]:
addDiscogs(mdbmaps, "Top40", "a5e6e308d31db00836b85dec9a28fac6", "120009")    ### [lum]
### Dance!
### Dance!  ---> [['Goleo Vi/lum']]


addDiscogs(mdbmaps, "Top40", "9cc336f08daf6b60fdc7df51c4951cd7", "771237")    ### [Soren Huss]
### Ocean Of You
### Ocean Of You  ---> [['Nik & Jay & Soren Huss']]


addDiscogs(mdbmaps, "Top40", "2f8ef80aa737b08ccbf4a4f597272dd4", "585244")    ### [Jimmy Jorgensen]
### Stjernestov
### Stjernestov  ---> [['Jimmy Jorgensen, Chapper & Annika']]


addDiscogs(mdbmaps, "Top40", "9361a0dd987d763afb7ebda59733dd4b", "164681")    ### [Michael Andr]
### Mad World
### Mad World  ---> [['Michael Andrews & Gary Jules', 'Adam Lambert', 'Gary Jules With Michael Andr', 'Michael Parsberg']]


addDiscogs(mdbmaps, "Top40", "8c4759f7357435c07209f893bd031627", "500375")    ### [Goleo Vi]
### Dance!
### Dance!  ---> [['Goleo Vi/lum']]


addDiscogs(mdbmaps, "Top40", "df5925386d5e772c54fab9c8ee4afc1a", "2449670")    ### [Bjornskov]
### Dimitto (Let Go)
### Dimitto (Let Go)  ---> [['Kato, Safri Duo & Bjornskov']]


addDiscogs(mdbmaps, "Top40", "ea6d8ca839c5321096bf52075b5913a3", "1091530")    ### [Blaes Bukki]
### Ga Vaek
### Ga Vaek  ---> [['Jokeren & Blaes Bukki']]


addAllMusic(mdbmaps, "Top40", "dff7682d6b7f9e652467f1d387940420", "mn0002116244")   ### [Michael Caro]
### Pas Pa Mikkel
### Pas Pa Mikkel  ---> [['Paprika Steen & Michael Caro']]


addDiscogs(mdbmaps, "Top40", "8293b2b928ccdcfda3a5b282235ed4ba", "489717")    ### [L.o.c.]
### Hospital
### Hospital  ---> [['Nephew & L.o.c.']]


addAllMusic(mdbmaps, "Top40", "f101734989e2fb6b8dd4f0192a44424e", "mn0001546498")   ### [Sha]
### Verdammt Ich Lieb Dich
### Verdammt Ich Lieb Dich  ---> [['Sha']]


addDiscogs(mdbmaps, "Top40", "c88659abef342c45aaeee57a7208f24b", "3871612")    ### [Jimillian]
### Slem Igen
### Slem Igen  ---> [['Blak, Jimillian & Ceci Luca']]


addDiscogs(mdbmaps, "Top40", "cde97ecc04c4fb0ab06efcceb6ba1766", "393367")    ### [Monday]
### Tuesday
### Tuesday  ---> [['I Love Makonnen & Drake', 'Burak Yeter & Danelle Sandoval', 'Monday/turning Japanese']]


addDiscogs(mdbmaps, "Top40", "f7a9e3bb639bf960c442d3854cc8e7f7", "709641")    ### [Herrelandsholdet]
### Hele Danmark Op At Sta
### Hele Danmark Op At Sta  ---> [['Thomas Helmig & Herrelandsholdet']]


mdbmaps["Top40"].save()

In [ ]:


addDiscogs(mdbmaps, "Top40", "8869a02f414bf8aa44ffa2491e057c18", "5690424")    ### [Sile Seoige]
### Maybe This Christmas  ,  Alabama & Friends  ,  The Breeze - An Appreciation Of JJ Cale
### Maybe This Christmas  ---> [['Sile Seoige & Friends']]
### Alabama & Friends  ---> [['Alabama & Friends']]
### The Breeze - An Appreciation Of JJ Cale  ---> [['Eric Clapton & Friends']]


addAllMusic(mdbmaps, "Top40", "23766e9564639a78dd2aa4469c656e33", "mn0000186917")   ### [Gabriella]
### Breaking Free
### Breaking Free  ---> [['Zac Efron & Vanessa Anne Hudgens', 'Troy & Gabriella']]


addAllMusic(mdbmaps, "Top40", "ee06f338805166cc652a5ffa29bfcaa4", "mn0000792831")   ### [Shane Macgowan]
### Dirty Old Town/road To Paradise
### Dirty Old Town/road To Paradise  ---> [['Jimmy Johnstone & Shane Macgowan']]


addDiscogs(mdbmaps, "Top40", "338160050dd8760f6e21fd314da0c262", "282163")    ### [Glen Keating]
### The Munster Song
### The Munster Song  ---> [['Glen Keating & Greg Ryan']]


addDiscogs(mdbmaps, "Top40", "8f7bba2ad693b7bd7a54e9f5561df5fd", "670431")    ### [Redone]
### Kick Ass
### Kick Ass  ---> [['Mika Vs. Redone']]


addAllMusic(mdbmaps, "Top40", "61e0cad6ba67b77cf03b21bdcc855000", "mn0001072817")   ### [Ndubz]
### We Dance On
### We Dance On  ---> [['N-Dubz', 'Ndubz & Bodyrox']]


addAllMusic(mdbmaps, "Top40", "8c59920da3b9afafbe8064702b8ae417", "mn0000866772")   ### [Levada Louca]
### Telephone
### Telephone  ---> [['Lady Gaga', 'Levada Louca & Belo']]


addAllMusic(mdbmaps, "Top40", "f9902337667b007c4cb0bd56853d6f77", "mn0001499382")   ### [Don Mescall]
### Catch Your Fall
### Catch Your Fall  ---> [['Aslan & Don Mescall']]


addDiscogs(mdbmaps, "Top40", "3a78a66700126f076bc9880499d46098", "6754053")    ### [Theresa Rex]
### What I Like About You
### What I Like About You  ---> [['Jonas Blue & Theresa Rex']]


addAllMusic(mdbmaps, "Top40", "45ec0da62cf46f95d1d19d6477e15223", "mn0003492788")   ### [Lewisham & Greenwich NHS Choir]
### A Bridge Over You
### A Bridge Over You  ---> [['Lewisham & Greenwich NHS Choir']]


addAllMusic(mdbmaps, "Top40", "b3a0765a3d6a1f9b5f0218cc1a78167d", "mn0001585547")   ### [DJ Champion]
### No Heaven
### No Heaven  ---> [['DJ Champion']]


addAllMusic(mdbmaps, "Top40", "4101fb8a2e7d70d355cb78bb670cafb2", "mn0000615641")   ### [N Trance]
### Set You Free
### Set You Free  ---> [['Gary Allan', 'N Trance']]


addAllMusic(mdbmaps, "Top40", "d4afc32b214e23fb75edc998fa24f102", "mn0000116049")   ### [D'side]
### Stronger Together
### Stronger Together  ---> [["D'side"]]


addAllMusic(mdbmaps, "Top40", "0edfaa56ae9a7023ef89e6dc038b27e7", "mn0000556741")   ### [Marshals]
### Make Her Cry
### Make Her Cry  ---> [['Marshals']]


addDiscogs(mdbmaps, "Top40", "ef5682c651f7c5295bc121ad58f18381", "1627595")    ### [St. Julien]
### Just Because
### Just Because  ---> [["Jane's Addiction", 'St. Julien']]


addAllMusic(mdbmaps, "Top40", "50eab1669ad5105c900ba050d23f7b37", "mn0002085581")   ### [Ruth-anne]
### Take Me Away
### Take Me Away  ---> [['Stonebridge', 'DJ S.K.T.', 'Fefe Dobson', 'Ruth-anne']]


addAllMusic(mdbmaps, "Top40", "123b17da9b4f0bc068b6211be4ba6eeb", "mn0002334760")   ### [Erick]
### The Beat Is Rockin
### The Beat Is Rockin  ---> [['Erick']]


addDiscogs(mdbmaps, "Top40", "0f4330d2f262dae9ff367c6c07b07ebd", "1551192")    ### [Temper Trap]
### Sweet Disposition
### Sweet Disposition  ---> [['Temper Trap']]


addAllMusic(mdbmaps, "Top40", "08de644904d4cf3503bdb90e1095eb0a", "mn0003330733")   ### [High Hopes Choir]
### High Hopes
### High Hopes  ---> [['Bruce Springsteen', 'Panic! At The Disco', 'Kodaline', 'High Hopes Choir']]


addAllMusic(mdbmaps, "Top40", "1b7dbb15e26894ee42a4088cd348af5c", "mn0002840885")   ### [Frankie Sandford]
### Dreaming
### Dreaming  ---> [['Frankie Sandford ::: Calvin Goldspink']]


addAllMusic(mdbmaps, "Top40", "7eac99ffb3a009020ae826491a7328a2", "mn0002300128")   ### [Calvin Goldspink]
### Dreaming
### Dreaming  ---> [['Frankie Sandford ::: Calvin Goldspink']]


addAllMusic(mdbmaps, "Top40", "a937fec3aae407f22faf4d34080e3c53", "mn0001093965")   ### [Frank Skinner]
### 3 Lions
### 3 Lions  ---> [['The Lightning Seeds ::: David Baddiel ::: Frank Skinner']]


addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
### What's It Gonna Be
### What's It Gonna Be  ---> [['H2O & Platinum']]


addAllMusic(mdbmaps, "Top40", "2bb54463a4b2619f4cddb21626fd43f3", "mn0000443133")   ### [Donaeo]
### Linguo
### Linguo  ---> [['Giggs & Donaeo']]


addDiscogs(mdbmaps, "Top40", "5a56911227b3087c65fa0c64cfc22858", "7175745")    ### [Rani]
### Post Malone
### Post Malone  ---> [['Sam Feldt & Rani']]


mdbmaps["Top40"].save()

In [ ]:
### The Beat Is Rockin
### The Beat Is Rockin  ---> [['Erick']]


addAllMusic(mdbmaps, "Top40", "45ec0da62cf46f95d1d19d6477e15223", "mn0003492788")   ### [Lewisham & Greenwich NHS Choir]
### A Bridge Over You
### A Bridge Over You  ---> [['Lewisham & Greenwich NHS Choir']]


addAllMusic(mdbmaps, "Top40", "b3a0765a3d6a1f9b5f0218cc1a78167d", "mn0001585547")   ### [DJ Champion]
### No Heaven
### No Heaven  ---> [['DJ Champion']]


addDiscogs(mdbmaps, "Top40", "a937fec3aae407f22faf4d34080e3c53", "356575")    ### [Frank Skinner]
### 3 Lions
### 3 Lions  ---> [['The Lightning Seeds ::: David Baddiel ::: Frank Skinner']]


addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
### What's It Gonna Be
### What's It Gonna Be  ---> [['H2O & Platinum']]


addDiscogs(mdbmaps, "Top40", "2bb54463a4b2619f4cddb21626fd43f3", "1480769")    ### [Donaeo]
### Linguo
### Linguo  ---> [['Giggs & Donaeo']]


addAllMusic(mdbmaps, "Top40", "4f7cfb22245eeec66816fe0eb080c4bf", "mn0003885021")   ### [Deno]
### Motorola
### Motorola  ---> [['DaBeatfreakz, Dappy, Swarmz & Deno']]


mdbmaps["Top40"].save()

In [ ]:


addAllMusic(mdbmaps, "Top40", "4dc4635967dd790b19e225de6e2b4385", "mn0000064130")   ### [Farm]
### All Together Now 2004
### All Together Now 2004  ---> [['Farm']]


addAllMusic(mdbmaps, "Top40", "6b87e39cce0801277df7a8bf475abb13", "mn0003517722")   ### [Choir For The Invictus Games]
### Flesh And Blood
### Flesh And Blood  ---> [['Choir For The Invictus Games & Gareth Malone']]


### Linguo
### Linguo  ---> [['Giggs & Donaeo']]


### Motorola
### Motorola  ---> [['DaBeatfreakz, Dappy, Swarmz & Deno']]

addAllMusic(mdbmaps, "Top40", "3772bea79ec33ccc235bd5eb74ec4188", "mn0000662225")   ### [H2O]
### What's It Gonna Be
### What's It Gonna Be  ---> [['H2O & Platinum']]


addAllMusic(mdbmaps, "Top40", "00f7af4fbcdb6bf41a2d99427eb39040", "mn0000121918")   ### [Royal Ballet SINF]
### Radio 4 Uk Theme
### Radio 4 Uk Theme  ---> [['Royal Ballet SINF']]


addDiscogs(mdbmaps, "Top40", "d4ce2ba190a3dd9ab80321657e708d43", "177063")    ### [X-centric Sound System]
### Electric Voodooland
### Electric Voodooland  ---> [['X-centric Sound System']]


### The Beat Is Rockin
### The Beat Is Rockin  ---> [['Erick']]


addAllMusic(mdbmaps, "Top40", "4cdbce49730dd9a82095802ddd829d5a", "mn0001585547")   ### [Champion]
### No Heaven
### No Heaven  ---> [['Champion']]


addAllMusic(mdbmaps, "Top40", "7bbe2c068275310da92c4a0daff9559c", "mn0002171059")   ### [Dj X-change]
addDiscogs(mdbmaps, "Top40", "7bbe2c068275310da92c4a0daff9559c", "1261283")    ### [Dj X-change]
### Bedrock
### Bedrock  ---> [['Young Money', 'Dj X-change']]


addAllMusic(mdbmaps, "Top40", "92bd8e18a270dc116528add61986c11d", "mn0002904642")   ### [Les Academiciens 2012]
### Toi + Moi
### Toi + Moi  ---> [['Les Academiciens 2012']]


mdbmaps["Top40"].save()

In [ ]:
updateManDB(2)

In [ ]:
### Bailame  ,  Toy Enamorao
### Bailame  ---> [['Nacho']]
### Toy Enamorao  ---> [['Mozart La Para Featuring Sharlene & Nacho']]


### Destination Brooklyn
### Destination Brooklyn  ---> [['Vicious']]


### La Zalamera
### La Zalamera  ---> [['Chichi Peralta With Joe Vasconcelos']]


addAllMusic(mdbmaps, "Billboard", "c92bf5840ff6bb2d2e7dccc44027f1a9", "mn0003634310")   ### [Kame To Yama P]
### Senaka Goshi No Chance
### Senaka Goshi No Chance  ---> [['Kame To Yama P']]


addAllMusic(mdbmaps, "Billboard", "728ce8b498b021f6a18f8015cae724c9", "mn0003568007")   ### [Roy]
### Because I Met You
### Because I Met You  ---> [['Roy']]


addAllMusic(mdbmaps, "Billboard", "5f587e8c33121ea678ba5076725dab00", "mn0000079923")   ### [BOWEN]
### The Last
### The Last  ---> [['Aventura', 'BOWEN']]


### Gyakuten Lovers
### Gyakuten Lovers  ---> [['Ken Tackey']]


addDiscogs(mdbmaps, "Billboard", "97e3ffc33ddd97ef8a602e4c6c7d3c20", "6832105")    ### [Hello!Pro All Stars]
### Yeah Yeah Yeah
### Yeah Yeah Yeah  ---> [['Hello!Pro All Stars']]


addDiscogs(mdbmaps, "Billboard", "b542360acfa29fda855d5a625904f563", "2156710")    ### [MAYDAY!]
### South Of 5th
### South Of 5th  ---> [['MAYDAY!']]


### Defy Gravity
### Defy Gravity  ---> [['The Elovaters']]


### Uchouten Shooter
### Uchouten Shooter  ---> [['Matsuri Nine.']]


### Eternal Echoes: Songs And Dance For The Soul
### Eternal Echoes: Songs And Dance For The Soul  ---> [['Itzhak Perlman/Cantor Yitzchak Meir Helfgot']]


### Yo Quiero Volver
### Yo Quiero Volver  ---> [['Silvio Mora']]


addAllMusic(mdbmaps, "Billboard", "f696888bba09f9e501bff688818aac1a", "mn0000890250")   ### [Richie Ray]
### Bajo, Piano & Bongo
### Bajo, Piano & Bongo  ---> [['Ismael Miranda Featuring Bobby Valentín, Richie Ray & Roberto Roena']]


### Si El Mundo Se Acabara
### Si El Mundo Se Acabara  ---> [['Los de La Nazza Featuring Justin Quiles']]


addAllMusic(mdbmaps, "Billboard", "12046f133d6822ebfda2f9afe91c821d", "mn0000944105")   ### [GrupoMania]
### Escapate
### Escapate  ---> [['Elvis Crespo Featuring GrupoMania']]


### Midnight
### Midnight  ---> [['Coldplay', 'Ez El Ezeta Featuring Nengo Flow & Kevin Lyttle']]


addAllMusic(mdbmaps, "Billboard", "9283a588faa79f17fe72198ee155c316", "mn0000866747")   ### [Nengo Flow]
### Midnight
### Midnight  ---> [['Coldplay', 'Ez El Ezeta Featuring Nengo Flow & Kevin Lyttle']]


addAllMusic(mdbmaps, "Billboard", "531f8d269b2e285284de2c44dd165aa3", "mn0002942051")   ### [Kehshi Yonezu]
### Uchiage Hanabi
### Uchiage Hanabi  ---> [['Daoko X Kehshi Yonezu']]


### Dejando Huellas
### Dejando Huellas  ---> [['Anibal de Gracia y Sus Invitados']]


### Moonlight
### Moonlight  ---> [['Wilber Pan & Yuan Yawei']]


### Bailame Despacio
### Bailame Despacio  ---> [['Xantos']]


### The Furthest Distance
### The Furthest Distance  ---> [['Hwang ChiYeul']]


addAllMusic(mdbmaps, "Billboard", "ef7dd6b43a4bce6816ad6867e4c63c44", "mn0002124769")   ### [24 Hrs]
### 24 Horas
### 24 Horas  ---> [['24 Hrs']]


### Naminori
### Naminori  ---> [['Kakigoori']]


addDiscogs(mdbmaps, "Billboard", "b33ccfc0247067a88d845dd08f530640", "1168200")    ### [Tomoni Itano]
### Fuini
### Fuini  ---> [['Tomoni Itano']]


### Bo peep Bo peep
### Bo peep Bo peep  ---> [['T-ara']]


mdbmaps["Billboard"].save()

In [ ]:
### Cronica De Dos Grandes  ,  Cronica De Dos Grandes: Recuerdos Con Amor  ,  Cronica De Dos Grandes: Los Inicios De Nuestra Historia
### Cronica De Dos Grandes  ---> [['Bronco/Los Bukis']]
### Cronica De Dos Grandes: Recuerdos Con Amor  ---> [['Bronco/Los Bukis']]
### Cronica De Dos Grandes: Los Inicios De Nuestra Historia  ---> [['Bronco/Los Bukis']]


addAllMusic(mdbmaps, "Billboard", "e19591dbfe1d583649369f724631f1b4", "mn0000197424")   ### [V. Fernandez]
addDiscogs(mdbmaps, "Billboard", "e19591dbfe1d583649369f724631f1b4", "419130")    ### [V. Fernandez]
### Arriba El Norte...
### Arriba El Norte...  ---> [['V. Fernandez/R. Ayala']]


### Arriba El Norte...
### Arriba El Norte...  ---> [['V. Fernandez/R. Ayala']]


addAllMusic(mdbmaps, "Billboard", "543444d971cb40fef7e247880ffeaadb", "mn0002118225")   ### [La Mari De Chambao]
### Tu Recuerdo
### Tu Recuerdo  ---> [['Ricky Martin Featuring La Mari De Chambao Y Tommy Torres']]


addAllMusic(mdbmaps, "Billboard", "dbc425901b964c15d6eff367325175d7", "mn0000218127")   ### [Jose Manuel Zamacona]
### Pero Te Vas A Arrepentir
### Pero Te Vas A Arrepentir  ---> [['K-Paz De La Sierra With Jose Manuel Zamacona']]


addAllMusic(mdbmaps, "Billboard", "408b11da7372d4640bb46db1da5ec79e", "mn0002691747")   ### [America Sierra]
### Intentalo
### Intentalo  ---> [['3BallMTY Featuring El Bebeto y America Sierra', '3BallMTY']]


### Yo No Soy Un Monstruo
### Yo No Soy Un Monstruo  ---> [['Elvis Crespo Featuring Ilegales']]


addAllMusic(mdbmaps, "Billboard", "1578c18e6a609a7b64d4f14a585a285a", "mn0003145706")   ### [Fifth Harmony Or Don Omar]
### Sin Contrato
### Sin Contrato  ---> [['Maluma Featuring Fifth Harmony Or Don Omar & Wisin']]


addAllMusic(mdbmaps, "Billboard", "b645b9b04861b872f2d8f2c68469539c", "mn0000859026")   ### [Natalia LaFourcade]
### Nunca Es Suficiente
### Nunca Es Suficiente  ---> [['Los Angeles Azules Featuring Natalia LaFourcade']]


mdbmaps["Billboard"].save()

In [ ]:
### Want Her  ,  Pure Water (with Migos)  ,  100 Bands (feat. Quavo, 21 Savage, YG & Meek Mill)  ,  Baguettes in the Face (feat. NAV, Playboi Carti & A Boogie Wit da Hoodie)  ,  On GOD (with YG & Tyga feat. A$AP Ferg & A$AP Rocky)  ,  Ballin' (with Roddy Ricch)

### Audi.  ,  123 (with Murda Beatz)  ,  123  ,  Nephew (feat. Lil Pump)  ,  What I Please (feat. Denzel Curry)

### Fear Inoculum  ,  Pneuma  ,  Invincible  ,  7empest

### Uber Everywhere  ,  Skateboard P  ,  Skateboard P (feat. Big Sean)  ,  Ned Flanders (feat. A$AP Ferg)

addAllMusic(mdbmaps, "Spotify", "7c32f7dcea75c7b747e3d44af7249f04", "mn0003764768")   ### [Cordae]
### Kung Fu  ,  RNP (feat. Anderson .Paak)  ,  Broke As Fuck  ,  Gifted (feat. Roddy Ricch)

addAllMusic(mdbmaps, "Spotify", "93d40ec406e8cd08584044aef77ce34e", "mn0003495582")   ### [KAROL G]
### Tusa  ,  Ay, DiOs Mío!  ,  BICHOTA

### Costa Rica  ,  Costa Rica (feat. The Kid LAROI) - Remix

### Party Girl  ,  Party Girl (Remix)

### APESHIT  ,  BOSS

### Tommy Lee (feat. Post Malone)  ,  Stuntin' On You (feat. DaBaby)

### Take Back Home Girl (Feat. Tori Kelly)  ,  I Don't Know About You

### STUPID (feat. Yung Baby Tate)  ,  Daisy

addAllMusic(mdbmaps, "Spotify", "6448500ba3a8e14687a2c89139850ece", "mn0003901054")   ### [Baby Keem]
### ORANGE SODA  ,  hooligan

### Disrespectful  ,  Creep On Me

### Sway (feat. Quavo & Lil Yachty)

### Off Deez (with J. Cole)

### Gassed Up

### BROWN SKIN GIRL

### Uno

### Bitchuary

### On A Roll

### Wrong

### Undrunk

### Worth It

### Somebody Else

### You & Me

### Oh Lord

### The Race

### Taped up Heart (feat. Clara Mae)

addAllMusic(mdbmaps, "Spotify", "70141b4bd7de61d411668869f7f8589b", "mn0000006320")   ### [TOTO]
### Africa

### Romantic - NOTD Remix

### Carol of the Bells

### High Without Your Love

### The Race

### Monster Mash

### Tu Sabes Que Te Quiero

### Get to the Money

addAllMusic(mdbmaps, "Spotify", "48c2f30e2dd7fece4c2a09c92e7fe4d7", "mn0000025634")   ### [Anita Kerr Singers]
### Jingle Bell Rock

### Goosebumps

### Yellow Hearts

### Deep End

### Put Your Records On

### E-GIRLS ARE RUINING MY LIFE!


### Lets Link

### Dancing in My Room

### Dicked Down in Dallas

### Your New Boyfriend

### Amazing Grace

### Mad at Disney

### Backyard Boy

### Adderall (Corvette Corvette) - Remix

### Cruel Intentions

### GOODMORNINGTOKYO!

### All I Want - From "High School Musical: The Musical: The Series"

addDiscogs(mdbmaps, "Spotify", "874b18e15fe469bfaa4e731482b5c4be", "3287801")    ### [K CAMP]
### Lottery (Renegade)

### Skechers

addAllMusic(mdbmaps, "Spotify", "c853db070b9ad54bbc0f522437436bc1", "mn0001023757")   ### [Banda MS de Sergio Lizárraga]
### Qué Maldición

### Coffee

### Deep End Freestyle

### Need Me

### Stunnin' (feat. Harm Franklin)

### Helpless



mdbmaps["Spotify"].save()

In [ ]:
### Want Her  ,  Pure Water (with Migos)  ,  100 Bands (feat. Quavo, 21 Savage, YG & Meek Mill)  ,  Baguettes in the Face (feat. NAV, Playboi Carti & A Boogie Wit da Hoodie)  ,  On GOD (with YG & Tyga feat. A$AP Ferg & A$AP Rocky)  ,  Ballin' (with Roddy Ricch)
### Want Her  ---> [['Mustard']]
### Pure Water (with Migos)  ---> [['Mustard']]
### 100 Bands (feat. Quavo, 21 Savage, YG & Meek Mill)  ---> [['Mustard']]
### Baguettes in the Face (feat. NAV, Playboi Carti & A Boogie Wit da Hoodie)  ---> [['Mustard']]
### On GOD (with YG & Tyga feat. A$AP Ferg & A$AP Rocky)  ---> [['Mustard']]
### Ballin' (with Roddy Ricch)  ---> [['Mustard']]

### Audi.  ,  123 (with Murda Beatz)  ,  123  ,  Nephew (feat. Lil Pump)  ,  What I Please (feat. Denzel Curry)
### Audi.  ---> [['Smokepurpp']]
### 123 (with Murda Beatz)  ---> [['Smokepurpp']]
### 123  ---> [['Smokepurpp']]
### Nephew (feat. Lil Pump)  ---> [['Smokepurpp']]
### What I Please (feat. Denzel Curry)  ---> [['Smokepurpp']]

### Uber Everywhere  ,  Skateboard P  ,  Skateboard P (feat. Big Sean)  ,  Ned Flanders (feat. A$AP Ferg)
### Uber Everywhere  ---> [['MadeinTYO']]
### Skateboard P  ---> [['MadeinTYO']]
### Skateboard P (feat. Big Sean)  ---> [['MadeinTYO']]
### Ned Flanders (feat. A$AP Ferg)  ---> [['MadeinTYO']]


### Disrespectful  ,  Creep On Me
### Disrespectful  ---> [['21 Savage', 'GASHI']]
### Creep On Me  ---> [['GASHI']]


### Costa Rica  ,  Costa Rica (feat. The Kid LAROI) - Remix
### Costa Rica  ---> [['Bankrol Hayden']]
### Costa Rica (feat. The Kid LAROI) - Remix  ---> [['Bankrol Hayden']]

### Tommy Lee (feat. Post Malone)  ,  Stuntin' On You (feat. DaBaby)
### Tommy Lee (feat. Post Malone)  ---> [['Tyla Yaweh']]
### Stuntin' On You (feat. DaBaby)  ---> [['Tyla Yaweh']]

### STUPID (feat. Yung Baby Tate)  ,  Daisy
### STUPID (feat. Yung Baby Tate)  ---> [['Ashnikko']]
### Daisy  ---> [['Ashnikko']]

### APESHIT  ,  BOSS
### APESHIT  ---> [['The Carters']]
### BOSS  ---> [['The Carters']]

### Take Back Home Girl (Feat. Tori Kelly)  ,  I Don't Know About You
### Take Back Home Girl (Feat. Tori Kelly)  ---> [['Chris Lane']]
### I Don't Know About You  ---> [['Chris Lane']]

### Party Girl  ,  Party Girl (Remix)
### Party Girl  ---> [['StaySolidRocky']]
### Party Girl (Remix)  ---> [['StaySolidRocky']]

### Adderall (Corvette Corvette) - Remix
### Adderall (Corvette Corvette) - Remix  ---> [['Popp Hunna']]

### Deep End Freestyle
### Deep End Freestyle  ---> [['Sleepy Hallow']]

### Need Me
### Need Me  ---> [['J.I the Prince of N.Y']]

### Stunnin' (feat. Harm Franklin)
### Stunnin' (feat. Harm Franklin)  ---> [['Curtis Waters']]

### Helpless
### Helpless  ---> [['Phillipa Soo']]

### Backyard Boy
### Backyard Boy  ---> [['Claire Rosinkranz']]

### Lets Link
### Lets Link  ---> [['WhoHeem']]

### Mad at Disney
### Mad at Disney  ---> [['salem ilese']]

### E-GIRLS ARE RUINING MY LIFE!
### E-GIRLS ARE RUINING MY LIFE!  ---> [['CORPSE']]

### Goosebumps
### Goosebumps  ---> [['HVME']]

### Coffee
### Coffee  ---> [['Quinn XCII']]

### Deep End
### Deep End  ---> [['Fousheé']]

### Dancing in My Room
### Dancing in My Room  ---> [['347aidan']]

### Dicked Down in Dallas
### Dicked Down in Dallas  ---> [['Trey Lewis']]

### Your New Boyfriend
### Your New Boyfriend  ---> [['Wilbur Soot']]

### Amazing Grace
### Amazing Grace  ---> [['Andrew Andraos']]

### Put Your Records On
### Put Your Records On  ---> [['Ritt Momney']]

### Too Much Sauce (feat. Future & Lil Uzi Vert)
### Too Much Sauce (feat. Future & Lil Uzi Vert)  ---> [['DJ ESCO']]

### Cruel Intentions
### Cruel Intentions  ---> [['Delacey']]

### Skechers
### Skechers  ---> [['DripReport']]

addAllMusic(mdbmaps, "Spotify", "ead60268db963c273d2b5d666c34ef5c", "mn0003443300")   ### [VÉRITÉ]
### Somebody Else
### Somebody Else  ---> [['VÉRITÉ']]

addAllMusic(mdbmaps, "Spotify", "6d0bbb8db0d2122b10f15dc6e07dd0c1", "mn0003371039")   ### [Marc E. Bassy]
### You & Me
### You & Me  ---> [['Marc E. Bassy']]

### Oh Lord
### Oh Lord  ---> [['MiC LOWRY']]

### Taped up Heart (feat. Clara Mae)
### Taped up Heart (feat. Clara Mae)  ---> [['KREAM']]

### Romantic - NOTD Remix
### Romantic - NOTD Remix  ---> [['Stanaj']]

### High Without Your Love
### High Without Your Love  ---> [['Loote']]

### Sway (feat. Quavo & Lil Yachty)
### Sway (feat. Quavo & Lil Yachty)  ---> [['NexXthursday']]

### The Race
### The Race  ---> [['Tay-K', '22 Savage']]

### Get to the Money
### Get to the Money  ---> [['Chad Focus']]

### Tu Sabes Que Te Quiero
### Tu Sabes Que Te Quiero  ---> [['Chucho Flash']]


addAllMusic(mdbmaps, "Spotify", "187a9b1acebe60867145dcc096312cb8", "")   ### [Bobby "Boris" Pickett]
addDiscogs(mdbmaps, "Spotify", "187a9b1acebe60867145dcc096312cb8", "")    ### [Bobby "Boris" Pickett]
### Monster Mash
### Monster Mash  ---> [['Bobby "Boris" Pickett']]


addAllMusic(mdbmaps, "Spotify", "14daf890c1544fb7ce4eb3422ce01ddf", "")   ### [22 Savage]
addDiscogs(mdbmaps, "Spotify", "14daf890c1544fb7ce4eb3422ce01ddf", "")    ### [22 Savage]
### The Race
### The Race  ---> [['Tay-K', '22 Savage']]


addAllMusic(mdbmaps, "Spotify", "7e301f86bbb9e2e5a95c89463444a6d4", "")   ### [Mykola Dmytrovych Leontovych]
addDiscogs(mdbmaps, "Spotify", "7e301f86bbb9e2e5a95c89463444a6d4", "")    ### [Mykola Dmytrovych Leontovych]
### Carol of the Bells
### Carol of the Bells  ---> [['Mykola Dmytrovych Leontovych']]


addAllMusic(mdbmaps, "Spotify", "850333d944cb7731aa381f24f1ac6b8a", "")   ### [Lean Trap]
addDiscogs(mdbmaps, "Spotify", "850333d944cb7731aa381f24f1ac6b8a", "")    ### [Lean Trap]
### La Modelo
### La Modelo  ---> [['Ozuna', 'Lean Trap']]


addAllMusic(mdbmaps, "Spotify", "c5686ff1b88412a406250a5fdfb7d7db", "")   ### [JID]
addDiscogs(mdbmaps, "Spotify", "c5686ff1b88412a406250a5fdfb7d7db", "")    ### [JID]
### Off Deez (with J. Cole)
### Off Deez (with J. Cole)  ---> [['JID']]

### Worth It
### Worth It  ---> [['YK Osiris']]


addAllMusic(mdbmaps, "Spotify", "ce3c103c18f37437fb4f28bd1f2fc759", "")   ### [FLETCHER]
addDiscogs(mdbmaps, "Spotify", "ce3c103c18f37437fb4f28bd1f2fc759", "")    ### [FLETCHER]
### Undrunk
### Undrunk  ---> [['FLETCHER']]

addAllMusic(mdbmaps, "Spotify", "c403314702c6a6af3d0f9fec374c9107", "")   ### [Luh Kel]
addDiscogs(mdbmaps, "Spotify", "c403314702c6a6af3d0f9fec374c9107", "")    ### [Luh Kel]
### Wrong
### Wrong  ---> [['Luh Kel']]


addAllMusic(mdbmaps, "Spotify", "e0251f7d87571e7fe5ed02a60815cd54", "")   ### [Ashley O]
addDiscogs(mdbmaps, "Spotify", "e0251f7d87571e7fe5ed02a60815cd54", "")    ### [Ashley O]
### On A Roll
### On A Roll  ---> [['Ashley O']]


addAllMusic(mdbmaps, "Spotify", "c0bde33feb703015ca4d3d0ae1fc9585", "")   ### [Shordie Shordie]
addDiscogs(mdbmaps, "Spotify", "c0bde33feb703015ca4d3d0ae1fc9585", "")    ### [Shordie Shordie]
### Bitchuary
### Bitchuary  ---> [['Shordie Shordie']]


addAllMusic(mdbmaps, "Spotify", "644ec747591292fe2198244beb274e58", "")   ### [Ambjaay]
addDiscogs(mdbmaps, "Spotify", "644ec747591292fe2198244beb274e58", "")    ### [Ambjaay]
### Uno
### Uno  ---> [['Ambjaay']]


addAllMusic(mdbmaps, "Spotify", "51732044ce01bbd865643834bd0a348b", "")   ### [Blue Ivy]
addDiscogs(mdbmaps, "Spotify", "51732044ce01bbd865643834bd0a348b", "")    ### [Blue Ivy]
### BROWN SKIN GIRL
### BROWN SKIN GIRL  ---> [['Blue Ivy']]


addAllMusic(mdbmaps, "Spotify", "76e1c7316d38224af8b740f53a7ee3ec", "")   ### [Ant Saunders]
addDiscogs(mdbmaps, "Spotify", "76e1c7316d38224af8b740f53a7ee3ec", "")    ### [Ant Saunders]
### Yellow Hearts
### Yellow Hearts  ---> [['Ant Saunders']]


addAllMusic(mdbmaps, "Spotify", "4909cfb2b7dc19f8b0caccc07ed7d598", "")   ### [TOKYO’S REVENGE]
addDiscogs(mdbmaps, "Spotify", "4909cfb2b7dc19f8b0caccc07ed7d598", "")    ### [TOKYO’S REVENGE]
### GOODMORNINGTOKYO!
### GOODMORNINGTOKYO!  ---> [['TOKYO’S REVENGE']]


addAllMusic(mdbmaps, "Spotify", "0980eca80d39428c6b1a1c3e3ac0a0d6", "")   ### [Olivia Rodrigo]
addDiscogs(mdbmaps, "Spotify", "0980eca80d39428c6b1a1c3e3ac0a0d6", "")    ### [Olivia Rodrigo]
### All I Want - From "High School Musical: The Musical: The Series"
### All I Want - From "High School Musical: The Musical: The Series"  ---> [['Olivia Rodrigo']]


addAllMusic(mdbmaps, "Spotify", "82553cfa3979f20878dd3f49ed77aa59", "")   ### [Lil Adrei]
addDiscogs(mdbmaps, "Spotify", "82553cfa3979f20878dd3f49ed77aa59", "")    ### [Lil Adrei]
### Burberry faygo
### Burberry faygo  ---> [['Lil Adrei']]

### Gassed Up
### Gassed Up  ---> [['Nebu Kiniza']]


mdbmaps["Spotify"].save()

In [ ]:



addAllMusic(mdbmaps, "Billboard", "2c2d626291f5a21f6ff6d0eb4b08591a", "mn0000026475")   ### [Anais]
### Lo Que Son Las Cosas
### Lo Que Son Las Cosas  ---> [['Anais']]


addAllMusic(mdbmaps, "Billboard", "583b33f3d5470cfd594d323d314512f8", "mn0000727291")   ### [Martinez Cruz Presenta Los Super Reyes]
### El Regrso De Los Reyes
### El Regrso De Los Reyes  ---> [['Martinez Cruz Presenta Los Super Reyes']]


addAllMusic(mdbmaps, "Billboard", "617a2ae62c3fea39ef85a119f0a80993", "mn0002083163")   ### [Evelyn Rubio Con La Orquesta de Blues de Calvin Owens]
### Hombres
### Hombres  ---> [['Evelyn Rubio Con La Orquesta de Blues de Calvin Owens']]

mdbmaps["Billboard"].save()

In [ ]:
def getMyData(corr):
    mydata = du.get_data(corr, engine)
    mydata.values
    return mydata.values

In [ ]:
results = df["name"].apply(getMyData)

In [ ]:


addAllMusic(mdbmaps, "Billboard", "ad57b21dc5badb6ad4b3213c635a589a", "mn0001687279")   ### [Shojo Jidai]
### Mr. Taxi  ,  PAPARAZZI  ,  Oh!
### Mr. Taxi  ---> [['Shojo Jidai']]
### PAPARAZZI  ---> [['Shojo Jidai']]
### Oh!  ---> [['Shojo Jidai']]


addDiscogs(mdbmaps, "Billboard", "b1762e9e4757c67687c3bbe63b8d2d1a", "4484557")    ### [M`S]
### Angelic Angel  ,  Bokutachi Ha Hitotsu No Hikari
### Angelic Angel  ---> [['M`S']]
### Bokutachi Ha Hitotsu No Hikari  ---> [['M`S']]


addAllMusic(mdbmaps, "Billboard", "308451bef86fcd177e3faf211de994ce", "mn0003485703")   ### [Nacho]
### Bailame  ,  Toy Enamorao
### Bailame  ---> [['Nacho']]
### Toy Enamorao  ---> [['Mozart La Para Featuring Sharlene & Nacho']]


addAllMusic(mdbmaps, "Billboard", "3ac0bd577a90dee773f9b910895f7ffa", "mn0002114689")   ### [TAEYEON]
### I: The 1st Mini Album (EP)
### I: The 1st Mini Album (EP)  ---> [['TAEYEON']]


addAllMusic(mdbmaps, "Billboard", "33b92ef8ac7f8ce2d2431e961775682e", "mn0002445126")   ### [ZERO-G]
### Zero Gravity
### Zero Gravity  ---> [['ZERO-G']]


addAllMusic(mdbmaps, "Billboard", "a4b81a4494d07498a86dc1fd66b65680", "mn0003358989")   ### [Park Hyo Min (T-ara)]
### Sketch
### Sketch  ---> [['Park Hyo Min (T-ara)']]


addAllMusic(mdbmaps, "Billboard", "13a0d6e566cc35b83cf11e64cef52e5f", "mn0003562734")   ### [PIKOTARO]
### PPAP (Pen-Pineapple-Apple-Pen)
### PPAP (Pen-Pineapple-Apple-Pen)  ---> [['PIKOTARO']]


addAllMusic(mdbmaps, "Billboard", "79c1d5700483a888cef4cc3c96777392", "mn0001638625")   ### [George Donaldson]
### The Road
### The Road  ---> [['George Donaldson']]


addAllMusic(mdbmaps, "Billboard", "bc53a647a9083dd5b5669d1c977944c9", "mn0003572459")   ### [Karry Wang]
### Miss Of Sky Wheel
### Miss Of Sky Wheel  ---> [['Karry Wang']]


addAllMusic(mdbmaps, "Billboard", "bee91a30c653a9d5146bdce553318cf4", "mn0003636549")   ### [Hwang ChiYeul]
### The Furthest Distance
### The Furthest Distance  ---> [['Hwang ChiYeul']]


addAllMusic(mdbmaps, "Billboard", "49d0e513440170788747bd6b4b5b6468", "mn0003821825")   ### [(Game Music)]
### The Idolm@Ster Cinderella Girls Starlight Master 3
### The Idolm@Ster Cinderella Girls Starlight Master 3  ---> [['(Game Music)']]


addAllMusic(mdbmaps, "Billboard", "986fe93fe00284401d2834780e8f5c38", "mn0003023534")   ### [Leo Ieiri]
### Kimi Ga Kureta Natsu
### Kimi Ga Kureta Natsu  ---> [['Leo Ieiri']]


addAllMusic(mdbmaps, "Billboard", "c92bf5840ff6bb2d2e7dccc44027f1a9", "mn0003634310")   ### [Kame To Yama P]
### Senaka Goshi No Chance
### Senaka Goshi No Chance  ---> [['Kame To Yama P']]

addAllMusic(mdbmaps, "Billboard", "5f587e8c33121ea678ba5076725dab00", "mn0000079923")   ### [BOWEN]
### The Last
### The Last  ---> [['Aventura', 'BOWEN']]


addAllMusic(mdbmaps, "Billboard", "1f6381c06850170b54c95e6a596ffb28", "mn0003782313")   ### [Ken Tackey]
### Gyakuten Lovers
### Gyakuten Lovers  ---> [['Ken Tackey']]


addAllMusic(mdbmaps, "Billboard", "b542360acfa29fda855d5a625904f563", "mn0000602072")   ### [MAYDAY!]
### South Of 5th
### South Of 5th  ---> [['MAYDAY!']]


addAllMusic(mdbmaps, "Billboard", "75d13abe72774129842a4beff75739ea", "mn0003598383")   ### [The Elovaters]
### Defy Gravity
### Defy Gravity  ---> [['The Elovaters']]


addAllMusic(mdbmaps, "Billboard", "d03d6af3ef8c0c4d1e104fe65d94cfb0", "mn0003671497")   ### [Matsuri Nine.]
### Uchouten Shooter
### Uchouten Shooter  ---> [['Matsuri Nine.']]


addAllMusic(mdbmaps, "Billboard", "3932c5031c30efcdfff5f6dea5b44452", "mn0002532879")   ### [Silvio Mora]
### Yo Quiero Volver
### Yo Quiero Volver  ---> [['Silvio Mora']]


addAllMusic(mdbmaps, "Billboard", "cb0c429ff2c1e5ce5f122fd3bc3ba9c7", "mn0002807645")   ### [T-ara]
### Bo peep Bo peep
### Bo peep Bo peep  ---> [['T-ara']]


addAllMusic(mdbmaps, "Billboard", "ae1cfe75bb6f91da7d223452168a3d59", "mn0000199162")   ### [Vicious]
### Destination Brooklyn
### Destination Brooklyn  ---> [['Vicious']]


mdbmaps["Billboard"].save()

In [ ]:
### My Fire  ,  You're Good For Me
### My Fire  ---> [['Nile Rodgers ::: Tony Moran ::: Kimberly Davis']]
### You're Good For Me  ---> [['Tony Moran Featuring Kimberly Davis']]


### Your Love / You're My Magician
### Your Love / You're My Magician  ---> [['Lime']]


addAllMusic(mdbmaps, "Billboard", "195f8a2ac2b277d5cf27f61ec5c8522b", "mn0002824115")   ### [Alex Newell]
### Kill The Lights
### Kill The Lights  ---> [['Luke Bryan', 'Alex Newell, Jess Glynne & DJ Cassidy With Nile Rodgers']]


addAllMusic(mdbmaps, "Billboard", "afd9a848c490feee0c8499aa7adb1366", "mn0000471277")   ### [Octahvia]
### In My Life
### In My Life  ---> [['Jose Nunez Featuring Octahvia']]


addAllMusic(mdbmaps, "Billboard", "2d8452e2e9bafdb5d6ecc1164a9c1e9e", "mn0002557064")   ### [Noferini & Marini]
### Push N Pull
### Push N Pull  ---> [['Noferini & Marini ::: Sylvia Tosun']]


addAllMusic(mdbmaps, "Billboard", "8a571837bf5781d909089200189f4239", "mn0001060976")   ### [Negin]
### Derezzed (2014)
### Derezzed (2014)  ---> [['Daft Punk Featuring Negin']]


addAllMusic(mdbmaps, "Billboard", "dfdf4276ed19e1bee6980849073e5c72", "mn0003409359")   ### [La Bagatelle Magique]
### Love Is Free (EP)
### Love Is Free (EP)  ---> [['Robyn & La Bagatelle Magique']]


### One Night
### One Night  ---> [['WTS Featuring Gia']]


addAllMusic(mdbmaps, "Billboard", "6b1a6a5b78128729c7aa17ddfa579fa3", "mn0000501147")   ### [Erick Moore]
### The New Anthem
### The New Anthem  ---> [['Reel 2 Real Featuring Erick Moore', 'N-Joi Featuring Mark Franklin And Nigel Champion']]


addAllMusic(mdbmaps, "Billboard", "6f08d78d2ac3d0cf7435b6ac57be2601", "mn0002765908")   ### [Raquel Castro]
### Places
### Places  ---> [['Xenia Ghali Featuring Raquel Castro']]


addAllMusic(mdbmaps, "Billboard", "5c60230b1e4d8427cad625838e43c659", "mn0003624727")   ### [Megn]
### Ghosting
### Ghosting  ---> [['Joe Bermudez Featuring Megn']]


### Testify!
### Testify!  ---> [['HiFi Sean Featuring Crystal Waters']]


addAllMusic(mdbmaps, "Billboard", "1b9b9c2c9c69661becf71e548b815e4c", "mn0001971972")   ### [The Dap-Kings Horns]
### Woman
### Woman  ---> [['Jill Scott', 'Justice', 'Kesha Featuring The Dap-Kings Horns']]


addAllMusic(mdbmaps, "Billboard", "0c049402f4ad27e4490b18fd3e8c92c0", "mn0003709976")   ### [CLiQ]
### Wavey
### Wavey  ---> [['CLiQ Featuring Alika']]


addDiscogs(mdbmaps, "Billboard", "867a419d3d571918c7aa14ff5aec1695", "5962444")    ### [Alika]
### Wavey
### Wavey  ---> [['CLiQ Featuring Alika']]


addAllMusic(mdbmaps, "Billboard", "f47e417f8ca6cd73ad2db432eb25a874", "mn0000697993")   ### [Zeitia]
### Slide On The Rhythm
### Slide On The Rhythm  ---> [['Arizona Featuring Zeitia']]


addAllMusic(mdbmaps, "Billboard", "4fb2a03482108c263e19b7116673a158", "mn0000300436")   ### [Little Louie]
### Ride On The Rhythm
### Ride On The Rhythm  ---> [['Little Louie/Marc Anthony']]


### Do It Again/Billie Jean
### Do It Again/Billie Jean  ---> [['Slingshot']]


### Found Love
### Found Love  ---> [['Double Dee Featuring Dany']]


addDiscogs(mdbmaps, "Billboard", "71a0fb17060a1366866a4f0a3bb0c032", "546073")    ### [Eleanor]
### Adventure
### Adventure  ---> [['Eleanor', 'Madeon']]


addDiscogs(mdbmaps, "Billboard", "05f7200789ce977849bcdece47aa5712", "49530")    ### [Narada]
### Divine Emotions
### Divine Emotions  ---> [['Narada']]


addDiscogs(mdbmaps, "Billboard", "f98f185812019c566c1f6538003d739b", "13092")    ### [Maurice]
### This Is Acid
### This Is Acid  ---> [['Maurice']]


addDiscogs(mdbmaps, "Billboard", "0cece9e0dc7e53c9b123669bc8dec8e1", "37227")    ### [D:REAM]
### U R The Best Thing
### U R The Best Thing  ---> [['D:REAM']]


addDiscogs(mdbmaps, "Billboard", "c5ee6743c389173d6fd28548e8a49868", "7286")    ### [The Goodmen]
### Give It Up
### Give It Up  ---> [['The Goodmen', 'Victor Calderone', 'Kevin Aviance']]


addAllMusic(mdbmaps, "Billboard", "0bd34daaeb33f068b0a7d5ad2b9cdae7", "mn0000050725")   ### [Sound Factory]
### Good Time
### Good Time  ---> [['Alan Jackson', 'Sound Factory', 'Owl City & Carly Rae Jepsen']]


addAllMusic(mdbmaps, "Billboard", "f2930bad4381d14646951630fe161c1e", "mn0000563805")   ### [Dajae]
### Day By Day
### Day By Day  ---> [['Dajae', 'Yolanda Adams']]


### Back To Basics
### Back To Basics  ---> [['Christina Aguilera', 'Shape: UK']]


addAllMusic(mdbmaps, "Billboard", "a44d3d8b405dab6dc5dd822d77dfa2b1", "mn0002110730")   ### [Peter Karr]
### Pause: Espresso: Breakfast in Bed: Contempo Instrumentals
### Pause: Espresso: Breakfast in Bed: Contempo Instrumentals  ---> [['Peter Karr']]


addAllMusic(mdbmaps, "Billboard", "446f07f2afc9d52de8b376ab0a913c9c", "mn0003439974")   ### [Madisyn Taylor]
### Meditation For The Highly Sensitive Person (EP)
### Meditation For The Highly Sensitive Person (EP)  ---> [['Madisyn Taylor']]


addAllMusic(mdbmaps, "Billboard", "a2a8887d0f49419d3c8c7e5708dbbeaa", "mn0000989224")   ### [Double Dee]
### Found Love
### Found Love  ---> [['Double Dee Featuring Dany']]


addAllMusic(mdbmaps, "Billboard", "ebf3c5677f4777481af3471ddaf94ac8", "mn0001032701")   ### [KLAAS]
### I've Been Thinking About You
### I've Been Thinking About You  ---> [['Londonbeat', 'KLAAS & Londonbeat']]


mdbmaps["Billboard"].save()

In [ ]:
cad.getAlbumsData("Found Love")

In [ ]:
updateMultiDB(2)

# Renames

In [93]:
toget, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)
analyzeRenames(manDB, dbRenames)

Artist ID [4448284] is not a member of artistIDToName.
Artist ID [5918943] is not a member of artistIDToName.
Artist ID [7765128] is not a member of artistIDToName.
Artist ID [8403120] is not a member of artistIDToName.
Artist ID [1444643] is not a member of artistIDToName.
Artist ID [12896823] is not a member of artistIDToName.
Artist ID [0003895005] is not a member of artistIDToName.
Artist ID [50258612] is not a member of artistIDToName.
Artist ID [11922823] is not a member of artistIDToName.
Artist ID [5454687] is not a member of artistIDToName.
Artist ID [262833] is not a member of artistIDToName.
Artist ID [0003635492] is not a member of artistIDToName.
Artist ID [0003658500] is not a member of artistIDToName.
Artist ID [13025953] is not a member of artistIDToName.
Artist ID [7351232] is not a member of artistIDToName.
Artist ID [11136556] is not a member of artistIDToName.
Artist ID [5817388] is not a member of artistIDToName.
Artist ID [13139755] is not a member of artistIDToNa

In [98]:
toget

{'Deezer': ['DENNIS',
  'Mc Livinho',
  'ANAVITÓRIA',
  'Mc Don Juan',
  'Jerry Smith',
  'Pineapple StormTv',
  'MC Kevin o Chris',
  'UM44K',
  'Hungria Hip Hop',
  'IZA',
  'WC no Beat',
  'Lexa',
  'Baco Exu do Blues',
  'Vitão',
  'MC JottaPê',
  'Grupo Menos É Mais',
  'MC Paulin da Capital',
  'Costa Gold',
  '1Kilo',
  'Mano Walter',
  'Pk',
  'Xand Avião',
  'Orochi',
  'Dj Guuga'],
 'AllMusic': ['Matuê', 'Luísa Sonza', 'Jonas Esticado', 'Niack']}

In [97]:
updateManDB(4)

========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7152 artist keys.
  There are currently 8338 renamed artist keys.
Trying to add 2 renamed artist keys
There are currently 7152 artist keys.
There are currently 8338 renamed artist keys.
There are currently 7153 artist keys.
There are currently 8340 renamed artist keys.
Saving 7153 artist keys to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
Saving data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 188.9kB.
Saved data to /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  --> This file is 188.9kB.
========================= masterArtistNameDB =========================
  Loading data from /Users/tgadfort/opt/anaconda3/envs/py37/musicnames/mainArtistNameDB.p
  There are currently 7153 artist keys.
  There are currently 83

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
updateManDB(4)

In [ ]:
cad.getAlbumsData("Rise")

In [ ]:
### The Sound Of Violence  ,  Together

### 1-2-3/Underestimate  ,  Heaven

### Aftermath (Here We Go)  ,  True Original

### BEAT OF THE NIGHT/PUMP IT UP

addDiscogs(mdbmaps, "Billboard", "f1be2026afee4969ba65b43956951fea", "4132584")    ### [Isha CoCo]
### Figure It Out

addAllMusic(mdbmaps, "Billboard", "1ecbb6c0ebd296028ec461fbbdd364fc", "mn0001264938")   ### [Dollar Man]
### Rock This Party (Everybody Dance Now)

addDiscogs(mdbmaps, "Billboard", "cde44749c67c2f94b8ac8785cf4321b0", "207447")    ### [U.D.A.U.F.L.]
### Most Precious Love

addDiscogs(mdbmaps, "Billboard", "07909094d91c8f75abeb5dd2d791da0d", "24058")    ### [G Club]
### Guitarra G

addAllMusic(mdbmaps, "Billboard", "a7de5b1f8914f72a61019db6794f6d8b", "mn0003162290")   ### [Rokelle]
### Take Me Away

mdbmaps["Billboard"].save()

In [ ]:
### Pakker Bar  ,  Blaest  ,  Al Fuego  ,  Gi Mig Det Hele  ,  JVB  ,  Demawa  ,  Mariah  ,  Opa  ,  Diana  ,  Super Mario  ,  De Snakker  ,  Lay Lay  ,  De Ved Godt  ,  Indifferent  ,  Gi Mig Det Hele  ,  Carnalismo  ,  For Mig Selv  ,  Limbo  ,  Oui  ,  Bang Bang  ,  Ingenting At Sige

addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]
### Positiv  ,  Boomerang  ,  74 Bars  ,  Herren Pa Trappan  ,  Glommer  ,  Igen  ,  Branschen  ,  Gflow  ,  Sverige Vet  ,  Ungefar  ,  Latch  ,  Klick

addDiscogs(mdbmaps, "Top40", "a9457f27a5cafd3e62b1d94169abdee7", "4403296")    ### [LX]
### Allstars  ,  Haifischnikez Allstars  ,  Super Lemon Haze  ,  Mercy Mercy  ,  Strassenbande  ,  Sa4  ,  Perdono  ,  The Party Is Over

addDiscogs(mdbmaps, "Top40", "81121caec82e95ee97bbaf3d3fd1a1a5", "3314194")    ### [SA4]
### Allstars  ,  Haifischnikez Allstars  ,  Super Lemon Haze  ,  Mercy Mercy  ,  Strassenbande  ,  Sa4  ,  Perdono  ,  The Party Is Over

addDiscogs(mdbmaps, "Top40", "9d7ee617eef7dfb58736ee3ca7c07865", "2126400")    ### [Jare]
### Haissa  ,  Levikset Repee  ,  Flexaa  ,  Keinutaan  ,  Lahtisitko  ,  Puistossa

addDiscogs(mdbmaps, "Top40", "1fb6ede6fc9981a8ebd58f26914da831", "208936")    ### [Remady]
### Give Me A Sign  ,  Single Ladies  ,  Holidays  ,  In My Dreams  ,  Another Day In Paradise  ,  Somebody Dance With Me

addAllMusic(mdbmaps, "Top40", "e2617599d1c934bd33657a28be1ca342", "mn0002851483")   ### [Kristo]
### The Race  ,  Ostavam Tuk  ,  Poveche Ot Vsichko  ,  Paradise  ,  No Mercy  ,  Igraem S Teb Do Kraja

addAllMusic(mdbmaps, "Top40", "9a5388731cb9a49315cbac42a140e44c", "mn0003979700")   ### [Tungevaag]
### Russian Roulette  ,  Wolf  ,  All For Love  ,  Wolf  ,  All For Love

addAllMusic(mdbmaps, "Top40", "8197f942e57571ed38f33aeecc423102", "mn0000904398")   ### [BossHoss]
### Don't Gimme That  ,  Do It  ,  Dos Bros  ,  Jolene  ,  Little Help

addAllMusic(mdbmaps, "Top40", "12ddcfb70159f15115286bd5290084b3", "mn0002763297")   ### [Raaban]
### Russian Roulette  ,  Wolf  ,  All For Love  ,  Wolf  ,  All For Love

addAllMusic(mdbmaps, "Top40", "8e00d7209419685ac3a1b0063cded4b7", "mn0002336077")   ### [DonkeyBoy]
### City Boy  ,  Ambitions  ,  Sometimes  ,  Broke My Eyes  ,  Downtown

addAllMusic(mdbmaps, "Top40", "0c5e04a1b9e4554f2b3ba0d5ba1a7cc0", "mn0003627665")   ### [Moeman]
### Op Je Hoede  ,  Intro  ,  Voorbij  ,  Hometown  ,  Merry

addAllMusic(mdbmaps, "Top40", "6ede0037b9522ac23b85661d71865f70", "mn0002897527")   ### [Jebroer]
### Banaan (Bigger Better Anthem)  ,  Kind Van De Duivel  ,  Engeltje  ,  Hoofd Omhoog  ,  Yellow Claw

addAllMusic(mdbmaps, "Top40", "70fd85554b55455d4dd4849de56b50ca", "mn0003291313")   ### [Ramiks]
### Culo  ,  Drup  ,  Mamacita  ,  Paraplu  ,  S/O Naar De...

addAllMusic(mdbmaps, "Top40", "8bbd146262d01da925cec72df725f3cf", "mn0001247159")   ### [Blockparty]
### HUTS  ,  SVP  ,  In Die Life  ,  Senorita  ,  Doekoe

addDiscogs(mdbmaps, "Top40", "2ce890ee581ea7a066f5d6a08ec31d78", "6305720")    ### [Joeyak]
### HUTS  ,  SVP  ,  In Die Life  ,  Senorita  ,  Doekoe

addAllMusic(mdbmaps, "Top40", "1dd23926a47382c2341a6700e13f6175", "mn0003763914")   ### [Young Ellens]
### HUTS  ,  SVP  ,  In Die Life  ,  Senorita  ,  Doekoe

addDiscogs(mdbmaps, "Top40", "ecf3b407fe3ed980bf938c0720cfcdde", "7100058")    ### [Mouad Locos]
### HUTS  ,  SVP  ,  In Die Life  ,  Senorita  ,  Doekoe

addAllMusic(mdbmaps, "Top40", "dcbb8aecd149b61a2b980573686c96c1", "mn0003556650")   ### [Jason Futuristic]
### Gass  ,  Gass  ,  Gass  ,  Tempo  ,  Senorita

addDiscogs(mdbmaps, "Top40", "0418314229f7ace6c62496671b4297d0", "3747415")    ### [Stepherd Skinto]
### Banaan (Bigger Better Anthem)  ,  Kind Van De Duivel  ,  Engeltje  ,  Hoofd Omhoog  ,  Yellow Claw

addAllMusic(mdbmaps, "Top40", "a2f3f9edf03e4bf85df289edf3084780", "mn0003934358")   ### [Dnoteondabeat]
### HotSpot  ,  Sprayad  ,  Nr 1  ,  Apoteket  ,  Ice Cream

addDiscogs(mdbmaps, "Top40", "ce3fbaee597ec5a524a308b812541f56", "848838")    ### [Rebecca]
### Bombay Dreams  ,  One Day  ,  Suddenly  ,  Bullets  ,  Taken Over

addAllMusic(mdbmaps, "Top40", "1e96d5c6acf8b95a736acf7aaeb52e50", "mn0003933006")   ### [Greekazo]
### HotSpot  ,  Sprayad  ,  Nr 1  ,  Apoteket  ,  Ice Cream

addDiscogs(mdbmaps, "Top40", "0893de3b7a69582832c2fdf20ca320ab", "5797329")    ### [Pablo Vittar]
### Sua Cara  ,  Downtown  ,  Vai Malandra  ,  Machika

addDiscogs(mdbmaps, "Top40", "d4ed085252376430421dd5d78c4198f2", "533880")    ### [Gerald De Palmas]
### Une Seule Vie (marcher Dans Le Sable)  ,  Tomber  ,  Elle S'ennuie  ,  Elle Danse Seule

addAllMusic(mdbmaps, "Top40", "914eb45aad9b04fb8952076e12986c6e", "mn0003333710")   ### [Mike & Colin]
### Gek Of Geniaal  ,  Mooi In Het Kwadraat  ,  Altijd Met Je Varen  ,  T Is Zomaar Zomer

addAllMusic(mdbmaps, "Top40", "4d15f5e5f09343d9d6e795618fa56677", "mn0002034863")   ### [Timo Raisanen]
### Lovers Are Lonely  ,  Let's Kill Ourselves A Son  ,  Sweet Marie  ,  Creep

addDiscogs(mdbmaps, "Top40", "e58a633be396b5286b60b11eafe27e8c", "831477")    ### [JUJU]
### Hardcore High  ,  Hi Babe  ,  Onnelliseksi  ,  Hullu

addDiscogs(mdbmaps, "Top40", "93ce511cfa464e9b26fa87db9cbc8934", "5157744")    ### [Hansie]
### Hey Meisje  ,  HUTS  ,  Vroeger  ,  Omhoog

addAllMusic(mdbmaps, "Top40", "57dd82cd58ebc1ee03495e7111f68837", "mn0003984725")   ### [Adel]
### Santa Lucia  ,  Kapabel 2  ,  No Cap  ,  SCI-FI

addAllMusic(mdbmaps, "Top40", "79b774a0cb6af992509de3574ce26315", "mn0003707565")   ### [Heuss Lenfoire]
### Khapta  ,  Ne Reviens Pas  ,  Dans Le Space  ,  Moulaga

addDiscogs(mdbmaps, "Top40", "cec35a4d1e39f4804f0329f8a891b635", "606524")    ### [Johnson]
### Hey Shorty  ,  Sjus  ,  Lucky Star  ,  One Love

addAllMusic(mdbmaps, "Top40", "c4b912d2b2efd4f0184e41a1103a803b", "mn0000220639")   ### [Priceless]
### Freak In The Weekend  ,  Rompe  ,  Wasteman  ,  Wow

addAllMusic(mdbmaps, "Top40", "cf3ec65c52b5c2ecf58d945c5fa0f4cb", "mn0001793545")   ### [Midenistis]
### Mia Zografia  ,  To Party De Stamata  ,  Se Enan Ticho  ,  S' Agapo

addAllMusic(mdbmaps, "Top40", "7efd2a63e16f752c9a12f9012b9344bb", "mn0000651709")   ### [Gert]
### Naar Zee  ,  Lieve Samson  ,  Heet  ,  Droom

addAllMusic(mdbmaps, "Top40", "b84e5beec1dd5837b0fcbb0c1bd35b65", "mn0002178429")   ### [Mokobe]
### Wesh  ,  C'est Dans La Joie  ,  Wakanda  ,  Getting Down

addAllMusic(mdbmaps, "Top40", "fcfbe1aa6e9e0638b7680e3af0b9a996", "mn0002616677")   ### [Livin R]
### Secret Sin!  ,  Inferno  ,  Easy  ,  You

addAllMusic(mdbmaps, "Top40", "938298238e1ff87fd67b5ff128f8b6d7", "mn0003901071")   ### [Marilena Animado]
### Secret Sin!  ,  Inferno  ,  Easy  ,  You

addAllMusic(mdbmaps, "Top40", "5a631e4252b9dc77c8a56308c85ce62f", "mn0001073548")   ### [Jandro]
### No Go Zone  ,  Wasteman  ,  Money Long

addAllMusic(mdbmaps, "Top40", "b4c90e7c25adeb8d32c2dca5137f3d00", "mn0003772188")   ### [Andry The Hitmaker]
### Mi Ami O No  ,  Parola  ,  Parola

addAllMusic(mdbmaps, "Top40", "78409e27f3ed3e90eab693c39a501ec3", "mn0003772189")   ### [Boro Boro]
### Lento  ,  Twerk  ,  Twerk

addAllMusic(mdbmaps, "Top40", "aad71a8e19e36f3255797a33800163ed", "mn0003775349")   ### [Mambolosco]
### Lento  ,  Twerk  ,  Twerk

addAllMusic(mdbmaps, "Top40", "77c4d723110af271b9457547a03b704c", "mn0000621368")   ### [Thastrom]
### Karlek Ar For Dom  ,  Hoghus-sång  ,  Ingenting Gor Mig

mdbmaps["Top40"].save()

In [ ]:
addAllMusic(mdbmaps, "Top40", "b44c3d5c561812173cb1fadc6bbadad0", "mn0000663567")   ### [Node]
### Pakker Bar  ,  Blaest  ,  Al Fuego  ,  Gi Mig Det Hele  ,  JVB  ,  Demawa  ,  Mariah  ,  Opa  ,  Diana  ,  Super Mario  ,  De Snakker  ,  Lay Lay  ,  De Ved Godt  ,  Indifferent  ,  Gi Mig Det Hele  ,  Carnalismo  ,  For Mig Selv  ,  Limbo  ,  Oui  ,  Bang Bang  ,  Ingenting At Sige

addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]
### Positiv  ,  Boomerang  ,  74 Bars  ,  Herren Pa Trappan  ,  Glommer  ,  Igen  ,  Branschen  ,  Gflow  ,  Sverige Vet  ,  Ungefar  ,  Latch  ,  Klick

addDiscogs(mdbmaps, "Top40", "ab418d4cdc49904fd3cb918b534f2e32", "6721597")    ### [Billy Hlapeto]
### Bashmaistorska  ,  Kogato Ti Triabvam  ,  Kakto Iskash  ,  Malkite Neshha  ,  Skochi Nad Men  ,  Kazhi Mi Veche Vsichko  ,  Zlatnite Momcheta  ,  Zapazeno Mjasto

addDiscogs(mdbmaps, "Top40", "60e0b849d26d3ca9fd7495fd7e7d6de6", "7630813")    ### [Dim4ou]
### Bashmaistorska  ,  Kogato Ti Triabvam  ,  Kakto Iskash  ,  Malkite Neshha  ,  Skochi Nad Men  ,  Kazhi Mi Veche Vsichko  ,  Zlatnite Momcheta  ,  Zapazeno Mjasto

addAllMusic(mdbmaps, "Top40", "81121caec82e95ee97bbaf3d3fd1a1a5", "mn0003373493")   ### [SA4]
### Allstars  ,  Haifischnikez Allstars  ,  Super Lemon Haze  ,  Mercy Mercy  ,  Strassenbande  ,  Sa4  ,  Perdono  ,  The Party Is Over

addAllMusic(mdbmaps, "Top40", "fb2419f9e6b98ed6e7dab91410e0edf1", "mn0002083163")   ### [Evelyn]
### One Night In Ibiza  ,  2012 (If The World Would End)  ,  Everybody  ,  Everybody  ,  Insomnia  ,  Sunshine (fly So High)  ,  Take Control

mdbmaps["Top40"].save()

In [ ]:

addAllMusic(mdbmaps, "Billboard", "113e6b6200dd4c2c1ef25d86a97afd0b", "mn0002565969")   ### [The Sparrow Quartet]
### Abigail Washburn & The Sparrow Quartet  ,  Bela Fleck & Abigail Washburn  ,  Echo In The Valley

addAllMusic(mdbmaps, "Billboard", "c8578e82fa1aeeb4a18f168aef04ae9e", "mn0002173979")   ### [SWR Baden-Baden and Freiburg Symphony Orchestra]
### Famous Overtures: Vol. III

addAllMusic(mdbmaps, "Billboard", "51294b506b8c6e69851a896f61733a5c", "mn0000807802")   ### [Eugue Groove]
### I Don't Mind

addAllMusic(mdbmaps, "Billboard", "045b97e9938944b95d7d9318ddf1c826", "mn0003759892")   ### [The Parallax Orchestra]
### Live At The Royal Albert Hall

addDiscogs(mdbmaps, "Billboard", "886e7e74ac9bf6882b98d6842466c5f6", "2055217")    ### [The Carducci Quartet O'Hara]
### Jethro Tull: The String Quartet

addAllMusic(mdbmaps, "Billboard", "df6827d2b0fefbd18ab39e2da672e485", "mn0003652638")   ### [Andrew Hawkley]
### Taboo

addAllMusic(mdbmaps, "Billboard", "32f5fe59dba72dd67515207d6c219bf1", "mn0002743475")   ### [Mountain Faith]
### That Which Matters

addAllMusic(mdbmaps, "Billboard", "8c053e9465b3d2832ec05ee2fce6cb70", "mn0001753298")   ### [Julian Vaughan]
### Traveling Mood

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "Top40", "eb97d29ea5e392c310e9afc1eead3d3c", "mn0000951374")   ### [Damian `Jr Gong` Marley]
addDiscogs(mdbmaps, "Top40", "eb97d29ea5e392c310e9afc1eead3d3c", "108897")    ### [Damian `Jr Gong` Marley]
### Make It Bun Dem

addAllMusic(mdbmaps, "Top40", "cffabc7a7118dfe493f15bf25fc920c5", "mn0003275915")   ### [DJ S.K.T]
### Take Me Away

### I'll Be There

addAllMusic(mdbmaps, "Top40", "44acadad22f6707e90956266bb8e4aa3", "mn0002306558")   ### [I Dream]
### Dreaming

addAllMusic(mdbmaps, "Top40", "fd1a9226e2acec385577624249df2364", "mn0003342371")   ### [Gareth Malone's All Star Choir]
### Wake Me Up

addDiscogs(mdbmaps, "Top40", "16bb746ce50cc703086005acad883267", "356575")    ### [Skinner]
### 3 Lions

addDiscogs(mdbmaps, "Top40", "e0b915bb964f72f2308f7a3a9e56548d", "356576")    ### [Baddiel]
### 3 Lions

### What's It Gonna Be

addAllMusic(mdbmaps, "Top40", "af3bd1548bef61d78989419ab8622b2d", "mn0003200964")   ### [Faul]
addDiscogs(mdbmaps, "Top40", "af3bd1548bef61d78989419ab8622b2d", "3287938")    ### [Faul]
### Changes

addAllMusic(mdbmaps, "Top40", "91eea18bc6000d5a8d99393e964f9475", "mn0002977260")   ### [Sam & The Womp]
addDiscogs(mdbmaps, "Top40", "91eea18bc6000d5a8d99393e964f9475", "2904452")    ### [Sam & The Womp]
### Bom Bom

addAllMusic(mdbmaps, "Top40", "f2fbf46d5a96d85fb92a3d8c599522cd", "mn0002421053")   ### [Rebekka Mathew]
### You Are The Music In Me

addAllMusic(mdbmaps, "Top40", "8cd433a14abc9f0cdb81579a138846ca", "mn0003330301")   ### [Sharaya J]
### Strip

addAllMusic(mdbmaps, "Top40", "5d318cf367cd5f1431aa65eee7377715", "mn0001482959")   ### [Rime]
### Yalla Habibi

addAllMusic(mdbmaps, "Top40", "ece00b4ab4febfc9fe6d81f7bc02a4df", "mn0002502193")   ### [Kaz Money]

addAllMusic(mdbmaps, "Top40", "ad4182c5c645774ce692eee97b4e1803", "mn0003912478")   ### [Darkoo]
### Gangsta

addAllMusic(mdbmaps, "Top40", "665b236b07ecd062386f0e23cdd2aa79", "mn0003666338")   ### [One Acen]
### Gangsta

addAllMusic(mdbmaps, "Top40", "6db9ba15066a9b5a40f9deabf66bd16b", "mn0003787834")   ### [Sidhu Moose Wala]
### Advice

addAllMusic(mdbmaps, "Top40", "0b62305e51334ddca2d85035dfd5661d", "mn0002480106")   ### [Liverpool Collective]
### Fields Of Anfield Road

addAllMusic(mdbmaps, "Top40", "907f4a93a96efba9f64cb3a2fa2e25fd", "mn0003444487")   ### [Dan Harkna]
### Sunshine

addAllMusic(mdbmaps, "Top40", "6b87e39cce0801277df7a8bf475abb13", "mn0003517722")   ### [Choir For The Invictus Games]
### Flesh And Blood

mdbmaps["Top40"].save()

In [ ]:
### Coming Home  ,  Airdrawn Dagger

### So Good To Me  ,  If U Got It

### Goodbye

addAllMusic(mdbmaps, "Top40", "7bbe2c068275310da92c4a0daff9559c", "mn0002171059")   ### [Dj X-change]
### Bedrock

addAllMusic(mdbmaps, "Top40", "5ace3ac65e8f500535d91a4dc0ee8a7d", "mn0000591277")   ### [Moony]
### Dove (i'll Be Loving You)

addAllMusic(mdbmaps, "Top40", "4cdbce49730dd9a82095802ddd829d5a", "mn0001585547")   ### [Champion]
### No Heaven

addAllMusic(mdbmaps, "Top40", "cfa8d6c438b06def554e5058982fe6c2", "mn0000992161")   ### [Smoot]
### Save A Life

addDiscogs(mdbmaps, "Top40", "92bd8e18a270dc116528add61986c11d", "7741420")    ### [Les Academiciens 2012]
### Toi + Moi

### Don't Rush

addDiscogs(mdbmaps, "Top40", "ddc98f5b85c9bbb653389887c34b4451", "11818")    ### [Ck]
### Dreamer

addDiscogs(mdbmaps, "Top40", "ff8dfeef6d100a42ea5ecdf64b41b0a5", "1781")    ### [The Jungle]
### Breathe Don't Stop

mdbmaps["Top40"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "617a2ae62c3fea39ef85a119f0a80993", "mn0002083163")   ### [Evelyn Rubio Con La Orquesta de Blues de Calvin Owens]
### Hombres

addAllMusic(mdbmaps, "Billboard", "13a0d6e566cc35b83cf11e64cef52e5f", "mn0003562734")   ### [PIKOTARO]
### PPAP (Pen-Pineapple-Apple-Pen)

addAllMusic(mdbmaps, "Billboard", "5f587e8c33121ea678ba5076725dab00", "mn0000079923")   ### [BOWEN]
### The Last

addAllMusic(mdbmaps, "Billboard", "1f0af2907fc6f9c9b589e1823c301ca3", "mn0002371302")   ### [Ricardo Delgado]
### Confidencias

addAllMusic(mdbmaps, "Billboard", "cd44d6b0cf601e36b17756b6f34504d2", "mn0001967293")   ### [Mario Mendoza]
### Mario Mendoza

addAllMusic(mdbmaps, "Billboard", "3c3679cd093ce31732563e9922eaa05d", "mn0003654273")   ### [Pedro Ramaya Beltran]
### Carnavaleando

addAllMusic(mdbmaps, "Billboard", "c142ef4784e3bbb3cbeeb9f5dbc0ef28", "mn0003289898")   ### [MAMAMOO]
### Purple (EP)

addAllMusic(mdbmaps, "Billboard", "67757f6bc60482eaeb89ced215fae495", "mn0000998391")   ### [Teddy Afro]
### Ethiopia

addAllMusic(mdbmaps, "Billboard", "d100531ed06428ac1314ad9d5225c22b", "mn0001578227")   ### [Cantores de Bayamon]
### Puerto Rico Es Una Fiesta

addAllMusic(mdbmaps, "Billboard", "07b54da0d93f193336ebef44ea0a7621", "mn0003576896")   ### [Chigualito]
### Son del Mar

addAllMusic(mdbmaps, "Billboard", "edaae0d81405cbae905736f20fd823cd", "mn0003619820")   ### [Young Shanty]
### Chalice Row Or Dig A Hole (EP)

addAllMusic(mdbmaps, "Billboard", "1c598980b0951e8c8364fcfdb2f89c24", "mn0003519238")   ### [Bryan Bautista]
### Promise

addAllMusic(mdbmaps, "Billboard", "3eb3416b6857910b9ab9d95358f5d9ca", "mn0003606180")   ### [Tenelle]
### For The Lovers

addAllMusic(mdbmaps, "Billboard", "c92bf5840ff6bb2d2e7dccc44027f1a9", "mn0003634310")   ### [Kame To Yama P]
### Senaka Goshi No Chance

addAllMusic(mdbmaps, "Billboard", "8042cba22260ceca125fedbc4a1a3868", "mn0003595367")   ### [J Sutta]
### Distortion

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "06a8a5a8878d609887f852796a1c1be5", "mn0001426855")   ### [Arcángel]
### Sentimiento, Elegancia & Maldad  ,  Tu Cuerpo Me Hace Bien  ,  Diosa de Los Corazones  ,  Zum Zum

addAllMusic(mdbmaps, "Billboard", "75cff7fefaa433e011c8220ebf4e64ab", "mn0001471849")   ### [Chino Y Nacho]
addDiscogs(mdbmaps, "Billboard", "75cff7fefaa433e011c8220ebf4e64ab", "2303802")    ### [Chino Y Nacho]
### Bla Bla Bla  ,  Ayantame  ,  Pasarla Bien  ,  Como Yo Te Quiero

addDiscogs(mdbmaps, "Billboard", "f8dfc1fd79fff7dac7e56da7dd6407d6", "4096148")    ### [JONGHYUN]
### Base: The First Mini Album (EP)  ,  Poet / Artist

addDiscogs(mdbmaps, "Billboard", "96fbdc37231074c2c40442378bfbd72f", "74514")    ### [J. Beeston]
### Pause: Karma: Clarity: Mind Inspiration  ,  Pause: Karma: Zen Piano: Dreamy Piano

addAllMusic(mdbmaps, "Billboard", "4096687a46c1090d4a07bc966950532d", "mn0000062200")   ### [Bernie Williams]
### Go For It  ,  Ritmo De Otono

addAllMusic(mdbmaps, "Billboard", "fb56ebfa359d8d878cfe52559026d4c4", "mn0000186398")   ### [Medeski]
### Combustication  ,  Juice

addAllMusic(mdbmaps, "Billboard", "275a28a865d406f5878bb09838204c63", "mn0001829369")   ### [The Innocent Criminals]
### Lifeline  ,  There Will Be A Light

addDiscogs(mdbmaps, "Billboard", "c8c8032a86f6780bbde11b13fe346707", "1882255")    ### [SHINee]
### Why So Serious?: The Misconceptions Of Me  ,  Odd: The 4th Album

addDiscogs(mdbmaps, "Billboard", "8c74712feea61f521462cfb1f9f84358", "416009")    ### [Richard Vission]
### Never Let Me Down  ,  I Like That

addAllMusic(mdbmaps, "Billboard", "ee50565d8fa46717417290944f586777", "mn0003268012")   ### [Yanfourd]
### Pirata De Amor  ,  Tracion

addAllMusic(mdbmaps, "Billboard", "78f9895f8ed99b4d9ed85035931b1c4d", "mn0001430923")   ### [ChocQuibTown]
### El Mar de Sus Ojos  ,  Desde El Dia En Que Te Fuiste

addAllMusic(mdbmaps, "Billboard", "d927c62ee30671a33566c39762772504", "mn0000228085")   ### [David Waxman]
### Ultra.Dance 05  ,  ThriveMix03

addDiscogs(mdbmaps, "Billboard", "b1762e9e4757c67687c3bbe63b8d2d1a", "4484557")    ### [M`S]
### Angelic Angel  ,  Bokutachi Ha Hitotsu No Hikari

addAllMusic(mdbmaps, "Billboard", "d4bd3b1cc12263b832c09f6674da1e57", "mn0002410530")   ### [Will Pan]
### Numb  ,  Dear Memories

addDiscogs(mdbmaps, "Billboard", "2e6ed79780f9a022cc9df118c9feb40e", "1121563")    ### [The Perry Twins]
### Bad, Bad Boy  ,  Euphoria

addAllMusic(mdbmaps, "Billboard", "dad59afd7476e763fa627dee584c6cba", "mn0000012430")   ### [Staxx Of Joy]
### Joy  ,  You

addAllMusic(mdbmaps, "Billboard", "1b08db14423fcb77398c831081bd0480", "mn0000602907")   ### [London Symphony (Williams)]
### Star Wars: A New Hope  ,  Star Wars: Return Of The Jedi

addAllMusic(mdbmaps, "Billboard", "1a437e23ad6fb8d3077e86589b52a1a3", "mn0000306033")   ### [Lucero]
### Siempre Contigo  ,  Un Lu* Jo

addAllMusic(mdbmaps, "Billboard", "aa2a8bfd76557f72e70937b086597934", "mn0000926469")   ### [Anthony]
### Yo Te Confieso  ,  What I Like

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "0545c0b8662f872c589afce4b6911f11", "mn0002658651")   ### [Tito "El Bambino" El Parton]
### Mi Cama Huele A Ti  ,  Diosa de Los Corazones  ,  Diosa de Los Corazones  ,  Pierdo La Cabeza  ,  Motivan2  ,  Ganas de Ti  ,  La Player (Bandolera)  ,  Hola  ,  Pierdo La Cabeza  ,  Motivan2  ,  Ganas de Ti  ,  La Player (Bandolera)  ,  Hola  ,  Otra Vez  ,  Otra Vez  ,  Te Quiero Pa'Mi  ,  Te Quiero Pa'Mi  ,  Subeme La Radio  ,  No Es Justo  ,  No Es Justo

addAllMusic(mdbmaps, "Billboard", "13ec547233780f5d33e353968527b9e3", "mn0000679314")   ### [Dave Aude]
### Make It Last  ,  I'm Gonna Get You  ,  Grass Is Greener  ,  Figure It Out  ,  Never Forget  ,  Something For The Weekend  ,  Hold Me  ,  Take Me Away  ,  Aftermath (Here We Go)  ,  True Original  ,  Hustlin'  ,  You Have To Believe  ,  Yeah Yeah 2017

addAllMusic(mdbmaps, "Billboard", "16e8a230f1803bf0b17e07f6e39225e3", "mn0000015398")   ### [Paul Brown]
### Winelite  ,  The Rhythm Method  ,  The Funky Joint  ,  Put It Where You Want It  ,  Hush  ,  Boogaloo  ,  Backstage Pass  ,  Piccadilly Circus

addAllMusic(mdbmaps, "Billboard", "e3a3e77a0ac7d47e00490703dfd2654b", "mn0003351662")   ### [Sandaime J Soul Brothers from EXILE TRIBE]
### R.Y.U.S.E.I  ,  O.R.I.O.N.  ,  Starting Over  ,  Summer Madness  ,  Happy?  ,  J.S.B. Dream

addAllMusic(mdbmaps, "Billboard", "babc8b7b1a3f743a72a29860c8ab8b3b", "mn0003074814")   ### [Sted-E]
### Synergy  ,  Believe  ,  Believe  ,  I Am House  ,  I Am House

addAllMusic(mdbmaps, "Billboard", "7b9d9672147379bba10f1441c814bb6a", "mn0002437438")   ### [El Potro Alvarez]
### Bla Bla Bla  ,  Ayantame  ,  Pasarla Bien  ,  Como Yo Te Quiero  ,  Se Acabo

addDiscogs(mdbmaps, "Billboard", "966db07a72b6d4eadc5ec1f7d9f3f6a2", "1337937")    ### [Archangel]
### Sentimiento, Elegancia & Maldad  ,  Tu Cuerpo Me Hace Bien  ,  Diosa de Los Corazones  ,  Zum Zum

addDiscogs(mdbmaps, "Billboard", "7d2fc8d4574b4f6972941478c183494e", "2435865")    ### [Bronco]
### Cronica De Dos Grandes  ,  Cronica De Dos Grandes: Recuerdos Con Amor  ,  Cronica De Dos Grandes: Los Inicios De Nuestra Historia

addAllMusic(mdbmaps, "Billboard", "ecd37430d9e0171902302d288c281f5c", "mn0000054266")   ### [Sus Canarios]
### Esta Llorando Mi Corazon  ,  Ardientes  ,  No Puedo Olvidarte

addAllMusic(mdbmaps, "Billboard", "99ccbacd8a76cd01ae7f124c403d252b", "mn0000066318")   ### [Paul Taylor]
### Burnin'  ,  Push To Start  ,  Supernova

addAllMusic(mdbmaps, "Billboard", "24a76c182c2a85684e7a0478715e5e87", "mn0000051740")   ### [Jose Carreras]
### A Celebration Of Christmas  ,  Merry Christmas From Vienna  ,  Notte Magica: A Tribute To Three Tenors

addAllMusic(mdbmaps, "Billboard", "88479115f3901acb361ca20c0c0af151", "mn0000071018")   ### [Ilegales]
### Yo No Soy Un Monstruo  ,  Ayantame  ,  Pasarla Bien

addAllMusic(mdbmaps, "Billboard", "883e69e0bb937cf784ae1a7c0587a63f", "mn0002410467")   ### [David Zonarosa Or Dyland]
### Balada (Tche Tcherere Tche Tche)  ,  El Taxi  ,  La Mala y La Buena

addAllMusic(mdbmaps, "Billboard", "2bdee05f12c51d1dfedb8d2ff06f5416", "mn0002376462")   ### [Terri B!]
### Big Fun  ,  Blind Heart  ,  Anthem Of House

addAllMusic(mdbmaps, "Billboard", "bca78c9f2ab3c9217fcbc4deb348ca02", "mn0003147793")   ### [The World]
### French Kiss  ,  Club Lonely  ,  Saved My Life

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "BillboardYE", "90983c7d7cb7b6c4e4443c4909116d16", "mn0000392428")   ### [Sus Bravos Del Norte]
### Antologia De Un Rey  ,  Antologia De Un Rey Vol. 2  ,  Leyenda Nortena: 30 Grandes Exitos

addAllMusic(mdbmaps, "BillboardYE", "011bd3e083e5725d5ad05892efd1004e", "mn0002184322")   ### [The Colorado Symphony]
### The Soft Bulletin: Recorded Live At Red Rocks Amphitheatre  ,  Live At Red Rocks  ,  Gregory Alan Isakov With The Colorado Symphony

addAllMusic(mdbmaps, "BillboardYE", "f19201e31420fee5b8b4b8d31afac756", "mn0000958533")   ### [Dave Brubeck]
### Time Out Featuring Take Five  ,  Bennett/Brubeck: The White House Sessions, Live 1962

addAllMusic(mdbmaps, "BillboardYE", "e3a5f3c7406f8572acc2f5d83bcdf7b4", "mn0000695943")   ### [Lennox]
### Motivan2  ,  Los Verdaderos

addAllMusic(mdbmaps, "BillboardYE", "0157a440c9d2d339120f95d28ea6dd03", "mn0000929220")   ### [Orchestra At Temple Square Jessop]
### Called To Serve  ,  Showtime! Music Of Broadway And Hollywood

addAllMusic(mdbmaps, "BillboardYE", "4ae32bbd2fe0e39284a6e3ad43d73669", "mn0003289856")   ### [Los Plebes del Rancho]
### El Karma  ,  Hablemos

addAllMusic(mdbmaps, "BillboardYE", "4fbbf7f930cb6537d418c785882a2b89", "mn0002261581")   ### [Yoncheva]
### George Frideric Handel: Messiah  ,  George Frideric Handel: Messiah, Highlights

addAllMusic(mdbmaps, "BillboardYE", "4e4dfbbafd57b7224bd51dc9ae473cd4", "mn0000017748")   ### [Patti LaBelle]
### The Gospel According to Patti LaBelle  ,  BEL Hommage

addAllMusic(mdbmaps, "BillboardYE", "c31265b36042cb96f154dec857ef762a", "mn0000929220")   ### [Orch. At Temple Square]
### George Frideric Handel: Messiah  ,  George Frideric Handel: Messiah, Highlights

addAllMusic(mdbmaps, "BillboardYE", "dd88591508a27fecc4b05c47f7d9698f", "mn0000806283")   ### [James F. Twyman]
### I Am: Wishes Fulfilled Meditation

addAllMusic(mdbmaps, "BillboardYE", "c7b1b183f35ee9f505e55de718ac37d6", "mn0003652039")   ### [The Dubplates]
### Box Full Of Steel

addAllMusic(mdbmaps, "BillboardYE", "149c5c27b9197ec6015f8231269aaefb", "mn0000767746")   ### [The Kentucky Ramblers]
### 40th Anniversary Celebration

addAllMusic(mdbmaps, "BillboardYE", "55932e14762136bb5cb432d6f40b97ae", "mn0000806126")   ### [Quicksilver]
### More Behind The Picture Than The Wall



mdbmaps["BillboardYE"].save()

In [ ]:

### The Gospel According to Patti LaBelle  ,  BEL Hommage

addDiscogs(mdbmaps, "BillboardYE", "8e89d63013e819158ff65ba347268796", "2985736")    ### [Milos Karadaglic]
### Pasion  ,  Mediterraneo

addDiscogs(mdbmaps, "BillboardYE", "45e10d4a89023cc6c3846948b33d7151", "1306656")    ### [Bayerisches Staatsorchester de Billy]
### L'Opera  ,  Nessun Dorma: The Puccini Album

addAllMusic(mdbmaps, "BillboardYE", "ae29bd5b08b9ddb2de43643a6455314c", "mn0002184322")   ### [The Colorado Symphony Gomes]
### Juno Concerto

addAllMusic(mdbmaps, "BillboardYE", "e074e7127d5f793fec562797097209d3", "mn0000239382")   ### [Academy Of St Martin In The Fields Denk]
### For The Love Of Brahms

addAllMusic(mdbmaps, "BillboardYE", "d9a2c6f1b91480a296005440ff6642da", "mn0000153653")   ### [Guest Orchestra: Utah Symphony]
### Mormon Tabernacle Choir & Friends

addAllMusic(mdbmaps, "BillboardYE", "a2d55d4e24fbfd30a55504f25823d9d5", "mn0000690837")   ### [Franz Welser-Most]
### Schubert: Symphony In C Major ("The Great") / Krenek: Static And Ecstatic

addAllMusic(mdbmaps, "BillboardYE", "437f8acd2190699691a9580009475be9", "mn0002228348")   ### [Pisaroni]
### Handel: Agrippina

mdbmaps["BillboardYE"].save()

In [ ]:
addAllMusic(mdbmaps, "BillboardYE", "74dc33ff69c1cb2f5284b058e4a4271b", "mn0003678368")   ### [Grupo Canaveral de Humberto Pabon]
addDiscogs(mdbmaps, "BillboardYE", "74dc33ff69c1cb2f5284b058e4a4271b", "3619794")    ### [Grupo Canaveral de Humberto Pabon]
### Juntos Por La Cumbia

addAllMusic(mdbmaps, "BillboardYE", "38b9f54e4d361e14ffea8c30fc37cc51", "mn0001647942")   ### [Bruckner Orchester Linz Conducted By Dennis Russell Davies]
### Glass: Symphony No. 9

addAllMusic(mdbmaps, "BillboardYE", "944fdb15a3576bba78d8467b0e662d12", "mn0002769031")   ### [Syreeta Thompson Trumpetlady]
### Evolution Of A Winner

addAllMusic(mdbmaps, "BillboardYE", "7f64551367c3181b7d052302680a15e0", "mn0003531694")   ### [Mau y Ricky]
### Para Aventuras y Curiosidades

addAllMusic(mdbmaps, "BillboardYE", "80e93e9d98e40b7b942073f7a710e118", "mn0000193772")   ### [Boston Symphony Orchestra Levine]
### Lorraine Hunt Lieberson Sings Peter Lieberson: Neruda Songs

addAllMusic(mdbmaps, "BillboardYE", "67be402b4ce838e8779f9eeb8d77c194", "mn0001826631")   ### [Roma Sinfonietta Orchestra Morricone]
### Yo-Yo Ma Plays Ennio Morricone

addAllMusic(mdbmaps, "BillboardYE", "eb33c715a0d6545882f7cea7835888d4", "mn0001661699")   ### [Sistine Chapel Choir Palombella]
### Veni Domine: Advent & Christmas At Sistine Chapel

addAllMusic(mdbmaps, "BillboardYE", "f5de6631de52a0f4ba7dcbbb9dc1c962", "mn0000350127")   ### [The Royal Philharmonic Concert Orchestra Mercurio]
### Sting: Live In Berlin

mdbmaps["BillboardYE"].save()



In [ ]:
addAllMusic(mdbmaps, "Top40", "996dc2db2e60fc3de7a14828df795171", "mn0003729938")   ### [Z. E.]

addAllMusic(mdbmaps, "Top40", "38cab1fb43cfee792ffbe27ee1a8b402", "mn0000534528")   ### [Helene Segara]

addDiscogs(mdbmaps, "Top40", "ab418d4cdc49904fd3cb918b534f2e32", "6721597")    ### [Billy Hlapeto]
### Bashmaistorska  ,  Kogato Ti Triabvam  ,  Kakto Iskash  ,  Malkite Neshha  ,  Skochi Nad Men  ,  Kazhi Mi Veche Vsichko  ,  Zlatnite Momcheta  ,  Zapazeno Mjasto

addDiscogs(mdbmaps, "Top40", "60e0b849d26d3ca9fd7495fd7e7d6de6", "7630813")    ### [Dim4ou]
### Bashmaistorska  ,  Kogato Ti Triabvam  ,  Kakto Iskash  ,  Malkite Neshha  ,  Skochi Nad Men  ,  Kazhi Mi Veche Vsichko  ,  Zlatnite Momcheta  ,  Zapazeno Mjasto

addAllMusic(mdbmaps, "Top40", "fa393c6150d3f872306dee987670e110", "mn0001462042")   ### [Keen`V]
### Dis-Moi Oui (Marina)  ,  Rien Qu'une Fois  ,  Le Son Qui Bam Bam  ,  Les Mots  ,  Ma Vie Au Soleil  ,  Keen V Elle T A Mata  ,  La Vie Du Bon Cote  ,  Petite Emilie

addAllMusic(mdbmaps, "Top40", "7061e79e486007acdc5409139bcfcc07", "mn0002526100")   ### [LaCrim]
### Pronto  ,  Sablier  ,  Mon Glock Te Mettra A Genoux  ,  Tout Le Monde Veut Des Loves  ,  Corleone  ,  Pocket Coffee  ,  Billets En Lair  ,  Grande Armee

addAllMusic(mdbmaps, "Top40", "81121caec82e95ee97bbaf3d3fd1a1a5", "mn0003373493")   ### [SA4]
### Allstars  ,  Haifischnikez Allstars  ,  Super Lemon Haze  ,  Mercy Mercy  ,  Strassenbande  ,  Sa4  ,  Perdono  ,  The Party Is Over

addAllMusic(mdbmaps, "Top40", "01849a34338bbba9d47201944a326596", "mn0002050239")   ### [Opposites]
### Broodje Bakpao  ,  Licht Uit  ,  Slapeloze Nachten  ,  Sukkel Voor De Liefde  ,  Slaap  ,  Brief Aan Jou  ,  Hey DJ

addAllMusic(mdbmaps, "Top40", "fb2419f9e6b98ed6e7dab91410e0edf1", "mn0002083163")   ### [Evelyn]
### One Night In Ibiza  ,  2012 (If The World Would End)  ,  Everybody  ,  Everybody  ,  Insomnia  ,  Sunshine (fly So High)  ,  Take Control


mdbmaps["Top40"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "89eab0113dd350e54cc04867d4cbcc04", "mn0000184098")   ### [John McDermott]
addDiscogs(mdbmaps, "Billboard", "89eab0113dd350e54cc04867d4cbcc04", "1436273")    ### [John McDermott]

addAllMusic(mdbmaps, "Billboard", "5dab93d7de7fdc41568fc96e979119e3", "mn0003740957")   ### [Dave Rowland]
### The Door Is Always Open  ,  Tear Time  ,  Golden Tears

addDiscogs(mdbmaps, "Billboard", "45908a9da68e8f594c1d999e8567a3bf", "4995184")    ### [John P. Kee & The New Life Community Choir]
### We Walk By Faith  ,  Blessed By Association  ,  Life And Favor

addAllMusic(mdbmaps, "Billboard", "5903a04a9d6f7e1f2434dd13def9c352", "mn0002960843")   ### [Nayer]
### Give Me Everything  ,  Hey Mama

addAllMusic(mdbmaps, "Billboard", "b66b1bfa1a8b0725ba1f139b87ad3a4c", "mn0000231468")   ### [Loon]
### I Need A Girl (Part One)  ,  I Need A Girl (Part Two)

addAllMusic(mdbmaps, "Billboard", "a6fbf98a79a048834d50ce68d67ad442", "mn0000621112")   ### [Mya]
### Case Of The Ex (Whatcha Gonna Do)  ,  Lady Marmalade

addAllMusic(mdbmaps, "Billboard", "832b5faffd094a5ab3b22f67d40db4b7", "mn0001049094")   ### [RKM]
### Masterpiece: Nuestra Obra Maestra

addAllMusic(mdbmaps, "Billboard", "7f2a3d27565fbcf78578a36856d29954", "mn0000403784")   ### [LeCrae]
### Live & Let Live

addAllMusic(mdbmaps, "Billboard", "3f4e690b965dfa3c6690758607cabd39", "mn0003542519")   ### [Vincent Bohanan]
### We Win: The Kingdom Declaration

addDiscogs(mdbmaps, "Billboard", "808c8b3a9909c7d7a3e5753792d1b995", "150433")    ### [LV]
### Gangsta's Paradise (From "Dangerous Minds")

addAllMusic(mdbmaps, "Billboard", "c346d5028767e3a4fda0260b4d1c1f97", "mn0000731951")   ### [Anita Wilson Accompanied By The Company]
### Sunday Song

addAllMusic(mdbmaps, "Billboard", "9d63b29d7ff8a3801b434b371008fef2", "mn0000454193")   ### [pureNRG]
### The Real Thing

addAllMusic(mdbmaps, "Billboard", "1bfaab8fb3874f792cc5510b2b95b028", "mn0003864623")   ### [Tobbi & Tommi]
### He Promised Me

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "BillboardYE", "d6316e7b2e17b6098435d0b89dfe8aeb", "mn0002568764")   ### [Jonathan Nelson & Purpose]
mdbmaps["BillboardYE"].removeArtistByKey("979a843a76ff54c4666958d6c0686732")
mdbmaps["BillboardYE"].removeArtistByKey("468693b2be1af905f93473f46b56f416")

mdbmaps["BillboardYE"].save()

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:
cad.getAlbumsData("George Frideric Handel: Messiah")

In [ ]:
updateManDB(2)

In [ ]:
addAllMusic(mdbmaps, "BillboardYE", "c271d8b907625e9a57e14b00578d3a13", "mn0003498068")   ### [Josh Baldwin]
### Stand In Your Love

addAllMusic(mdbmaps, "BillboardYE", "e6c4cc7277a55f68ba0a9553dfa64494", "mn0003492731")   ### [Micah Tyler]
### Never Been A Moment  ,  Amen

addAllMusic(mdbmaps, "BillboardYE", "6ea0ac7bf7679a09b82feecf9282260d", "mn0003870377")   ### [We The Kingdom]
### Holy Water  ,  God So Loved

addAllMusic(mdbmaps, "BillboardYE", "e5e41fbe40ec9cb306a44dd470d57909", "mn0002698695")   ### [Hope Darst]
### Peace Be Still

addAllMusic(mdbmaps, "BillboardYE", "1ac2db404a4530bbdc1312a2a537fb55", "mn0000281564")   ### [Mark Harris]
### Find Your Wings

addAllMusic(mdbmaps, "BillboardYE", "e82b70634bd408362fd21eb1ee84af77", "mn0000017048")   ### [tree63]
### All Over The World

addAllMusic(mdbmaps, "BillboardYE", "e9b3aa0450e6e7ef8c9f74c331224df4", "mn0000946806")   ### [newworldson]
### Learning To Be The Light  ,  There Is A Way

addAllMusic(mdbmaps, "BillboardYE", "e6d7745eb03517e668489d1ea3c7ce19", "mn0000979519")   ### [Echoing Angels]

addAllMusic(mdbmaps, "BillboardYE", "2828bf7a9fa1f4fd3f6eab39212ab82d", "mn0001562036")   ### [Jo Jo Jorge Falcon]

addAllMusic(mdbmaps, "BillboardYE", "9c369f58464c4334af94844fe486b803", "mn0003885860")   ### [Sunday Service]
### Jesus Is Born

addAllMusic(mdbmaps, "BillboardYE", "0c549661a6664c25879ca7886710ac23", "mn0003811002")   ### [Maverick City Music]
### Maverick City, Vol. 3: Part 1

addAllMusic(mdbmaps, "BillboardYE", "131e357fb035d7f65af3dd3a523332e9", "mn0001044136")   ### [The Lost Fingers]
### Lost In The 80's

addAllMusic(mdbmaps, "BillboardYE", "446a9112614084c5c98d9c8c3fcc70be", "mn0002114807")   ### [Dillon Chase]

addAllMusic(mdbmaps, "BillboardYE", "9c4b5988c62a8b61d08cf2f5255124d2", "mn0002687866")   ### [Madelyn Berry]
### Waymaker

addAllMusic(mdbmaps, "BillboardYE", "62000351594925ac0c8d2e7da8447458", "mn0002335388")   ### [Vanessa Campagna]
### Waymaker

addAllMusic(mdbmaps, "BillboardYE", "a44e538382b982fdcd999f67e958cf6f", "mn0000204077")   ### [Bart Millard]
### Words  ,  No Matter What

addDiscogs(mdbmaps, "BillboardYE", "7ba48fe209a75ce089a3f33b4e9fa63b", "2982228")    ### [The FGBCFMC]
### Cry Your Last Tear

addAllMusic(mdbmaps, "BillboardYE", "9ec2841d41a59e221a0b3497b334e2be", "mn0002586063")   ### [Larry Callahan]
### The Evolution II

addDiscogs(mdbmaps, "BillboardYE", "bbca5a23b3a285175d0ba3ec4c7bee30", "4995184")    ### [John P. Kee & The New Life Community Choir]
### Life And Favor

addAllMusic(mdbmaps, "BillboardYE", "f63d2bfca3bdfdb66dd01e6eb02a28a7", "mn0000772877")   ### [Bishop G.E. Patterson]
### Singing The Old Time Way Volume 2

mdbmaps["BillboardYE"].save()

In [ ]:
updateManDB(2)

In [ ]:


mdbmaps["BillboardYE"].save()

In [ ]:

### Move B***h

### I Know What You Want

### Beautiful

### Something Keeps Calling  ,  Never Give You Up

### Make The Most

### You & I

addAllMusic(mdbmaps, "BillboardYE", "3ce23050d8843b8d96558a85175f63b4", "mn0003262021")   ### [Swae Lee Or Rae Sremmurd]
### Sativa

### March

addAllMusic(mdbmaps, "BillboardYE", "daafe83a627a390818a4d84a1cc2b584", "mn0000103872")   ### [Jacoby Shaddix]
### Wolf Totem

### Lay Your Head On Me

addAllMusic(mdbmaps, "BillboardYE", "a6b2d85fd2bb6caa2dbbfb632d160254", "mn0002908858")   ### [Myles Kennedy And The Conspirators]
### Apocalyptic Love

### Say Hey (I Love You)

mdbmaps["BillboardYE"].save()

In [ ]:

### Ballin'  ,  High Fashion  ,  Pure Water

addDiscogs(mdbmaps, "BillboardYE", "5bcde9488392d14111cd5b222b2e788d", "5809783")    ### [BIA]
### BEST ON EARTH

addDiscogs(mdbmaps, "BillboardYE", "be70e0ca1af7931aab5f1cd67c6a8816", "7829104")    ### [THE SCOTTS]
### The Scotts

addAllMusic(mdbmaps, "BillboardYE", "c04022eafc2109a6517166349c2182b4", "mn0002901034")   ### [IamSu!]
### Gas Pedal

### White Walls

addAllMusic(mdbmaps, "BillboardYE", "98b48695d58c81efb7b5738dbd19b2e4", "mn0001530959")   ### [Casino]
### Move That Doh  ,  Mercy

### The Business

### UP!

addAllMusic(mdbmaps, "BillboardYE", "ef25c72d7bada61e1a171cfd4b4b5893", "mn0002550537")   ### [Skipper or 50 Cent]
### UP!

addAllMusic(mdbmaps, "BillboardYE", "f8ad87bec3bbfcedf98ecaea06cb1bbe", "mn0001981371")   ### [DTP]
### Ludacris Presents...Disturbing Tha Peace

addDiscogs(mdbmaps, "BillboardYE", "a938a487c4a5720222d91ce0a0a8e2e4", "133774")    ### [Lil Peanut]
### Lean Wit It, Rock Wit It

addAllMusic(mdbmaps, "BillboardYE", "72b04864ac1a5f066699a86dfd8d81c3", "mn0000574974")   ### [Kandi Girl]
### U And Dat

addDiscogs(mdbmaps, "BillboardYE", "de48606f253c34073057df92b65f39fa", "2178492")    ### [Kevin K-MAC McCall]
### Strip

### Some Cut

addAllMusic(mdbmaps, "BillboardYE", "4220ecc3c03ecc59f12a95e0671756ba", "mn0002540349")   ### [Roman GianArthur]
### Classic Man

addAllMusic(mdbmaps, "BillboardYE", "e93073e8cda20c7ad949d5e7c99e2e35", "mn0001735175")   ### [Jay Biz]
### Poppin'

mdbmaps["BillboardYE"].save()

In [ ]:
addAllMusic(mdbmaps, "BillboardYE", "e9e910c1481daa2e23ba7c16946fc033", "mn0003264096")   ### [Boosie BadAzz]
### Touch Down 2 Cause Hell

addAllMusic(mdbmaps, "BillboardYE", "a3b3b076b58dd330bfbb945bfd7484c3", "mn0000315463")   ### [Scarface]
### EMERITUS  ,  Made

### We Fly High  ,  Pop Champagne

addAllMusic(mdbmaps, "BillboardYE", "e0f84561e17208c8f8a47c000497073d", "mn0000192537")   ### [G Unit]
### Beg For Mercy

addAllMusic(mdbmaps, "BillboardYE", "473a63b53041c90fad0384d3e307d134", "mn0001451588")   ### [Avery*Sunshine]
### Call My Name

### Moral Of The Story

addAllMusic(mdbmaps, "BillboardYE", "094ced7f5000ffe909ce65ef1c36c530", "mn0003600918")   ### [flora cash]
### You're Somebody Else

mdbmaps["BillboardYE"].save()

In [ ]:
updateManDB(2)

In [ ]:
addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
### My Dogs

addAllMusic(mdbmaps, "Top40", "21da687c5162d957e0e8aaa39eff1e80", "mn0003750925")   ### [The Carters]
### Apes**t  ,  Everything Is Love

addAllMusic(mdbmaps, "Top40", "c00c9a5d2e3053716de9b79898cb5691", "mn0001225951")   ### [Mustard]
### Perfect Ten  ,  Ballin'

addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
### Hold Me

addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]
### Cry For Me

addAllMusic(mdbmaps, "Top40", "5a888aa5d5b0835533b2a73dd895b461", "mn0002528230")   ### [DCUP]
### We No Speak Americano

addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
### White Walls

addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
### The Hanging Tree

addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
### I Hate U, I Love U  ,  Lights Down Low

addAllMusic(mdbmaps, "Top40", "9d337e01e83fc2b9d9fcbd6930bfb107", "mn0003715465")   ### [Casper Magico]
### Te Bote  ,  Otro Trago

mdbmaps["Top40"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "f48a161c0063d474c2eab5dc2e5195cd", "mn0003442077")   ### [Thomas Miles aka Nephew Tommy]
### Presents: Prank Phone Calls Vol 5: Church Folks Gotta Laugh Too  ,  Presents: Prank Phone Calls: Church Folks Gotta Laugh Too Vol 2

addAllMusic(mdbmaps, "Billboard", "efee2dea144675a9daea006446b60011", "mn0002817738")   ### [Brian Brushwood Justin Robert Young]
### Night Attack 2: Enjoy The Garden  ,  Night Attack (Live)

addAllMusic(mdbmaps, "Billboard", "8a8f1ba4fa40b97993e05516b6cbd260", "mn0003232100")   ### [Deitrick Haddon's LXW (League Of Xtraordinary Worshippers)]
### Deitrick Haddon's LXW

addAllMusic(mdbmaps, "Billboard", "7ca86f084b469e552a767a3b582bed15", "mn0001081792")   ### [Swoope]
### Sinema

addAllMusic(mdbmaps, "Billboard", "84c774e5a41773f191b62bbc09774dab", "mn0003465054")   ### [Braiden Sunshine]
### Amazing Grace

addAllMusic(mdbmaps, "Billboard", "62fa94e3fb88cdeb88e50b1f2ae64cfe", "mn0000563196")   ### [Myron Butler & Levi]
### On Purpose

addAllMusic(mdbmaps, "Billboard", "514877cce2b057728c36e0c14a26f0bc", "mn0003586280")   ### [My Dad Wrote A Porno]
### My Dad Wrote A Christmas Porno

addAllMusic(mdbmaps, "Billboard", "3a799a523bc6e3e6e953894a176b7d97", "mn0003874962")   ### [Fresh Start Worship]
### Mention

addAllMusic(mdbmaps, "Billboard", "7f2a3d27565fbcf78578a36856d29954", "mn0000403784")   ### [LeCrae]
### Live & Let Live

addAllMusic(mdbmaps, "Billboard", "d8f8476e6164eabe4fb4ab3ba3f3b7d9", "mn0002558731")   ### [Suzy Rock]
### Fuego

addAllMusic(mdbmaps, "Billboard", "60021b6a1be2c390a233cce11db4b5b9", "mn0002867667")   ### [Them Idiots]
### Whirled Tour

addAllMusic(mdbmaps, "Billboard", "99c44622cf1137d8ffff21a66c3d3028", "mn0000175286")   ### [D. Parton]
### Higher Medley

addAllMusic(mdbmaps, "Billboard", "66694ccf7b6dfaed0f263e8f9569ef71", "mn0003288960")   ### [Dimitri McDowell]
addDiscogs(mdbmaps, "Billboard", "66694ccf7b6dfaed0f263e8f9569ef71", "4123531")    ### [Dimitri McDowell]
### Sweet Victory

addAllMusic(mdbmaps, "Billboard", "30f2dacc35fad2a476eb8db246f243f5", "mn0002447683")   ### [Mr. Talkbox]
### Feel It

addAllMusic(mdbmaps, "Billboard", "7008c483e43dff7585e75c0588dce00f", "mn0000494388")   ### [GabeReal]
### Eye Of The Storm

addAllMusic(mdbmaps, "Billboard", "4831b39973a020abf296356f3abfe702", "mn0001467056")   ### [Ryan Stevenson]
### Eye Of The Storm

addAllMusic(mdbmaps, "Billboard", "fc61047e5a0c49d67bc6c28515f740f6", "mn0003520491")   ### [Lindy Conant]
### Every Nation

addAllMusic(mdbmaps, "Billboard", "5cf648bdbb963bf9021e2ccfa0b5e811", "mn0003528162")   ### [The Circuit Riders]
### Every Nation

addAllMusic(mdbmaps, "Billboard", "31c5c26ecfd994d64599a8c74fcb0e0c", "mn0003595666")   ### [GEI]
### Hang On

addAllMusic(mdbmaps, "Billboard", "e7fdd33db429c91d51fbb11f05b6b708", "mn0002743423")   ### [Le'Andria]
### Grace

addAllMusic(mdbmaps, "Billboard", "3f4e690b965dfa3c6690758607cabd39", "mn0003662860")   ### [Vincent Bohanan]
### We Win: The Kingdom Declaration

addAllMusic(mdbmaps, "Billboard", "17ade4a4fbc717fa2101363ee7fd37c3", "mn0000178135")   ### [The Edwin Hawkins' Singers]
### Oh Happy Day

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "Billboard", "d6cac33cc3555e4e1f5ce0d2b3cae450", "mn0001481616")   ### [Connor Christian & Southern Gothic]
addDiscogs(mdbmaps, "Billboard", "d6cac33cc3555e4e1f5ce0d2b3cae450", "4558282")    ### [Connor Christian & Southern Gothic]
### New Hometown

addDiscogs(mdbmaps, "Billboard", "808c8b3a9909c7d7a3e5753792d1b995", "150433")    ### [LV]
### Gangsta's Paradise (From "Dangerous Minds")

addAllMusic(mdbmaps, "Billboard", "89eab0113dd350e54cc04867d4cbcc04", "mn0000184098")   ### [John McDermott]
addDiscogs(mdbmaps, "Billboard", "89eab0113dd350e54cc04867d4cbcc04", "1436273")    ### [John McDermott]
### The Irish Tenors: Home For Christmas  ,  The Irish Tenors: Live In Belfast  ,  The Irish Tenors: Ellis Island  ,  The Irish Tenors: Live In Belfast  ,  The Irish Tenors: Ellis Island

addDiscogs(mdbmaps, "Billboard", "265d96f09b9e09bd2ea57aa249d40bd6", "4995184")    ### [New Life]
### Blessed By Association

addDiscogs(mdbmaps, "Billboard", "b66b1bfa1a8b0725ba1f139b87ad3a4c", "92281")    ### [Loon]
### I Need A Girl (Part Two)  ,  I Don't Wanna Know

addAllMusic(mdbmaps, "Billboard", "832b5faffd094a5ab3b22f67d40db4b7", "mn0001049094")   ### [RKM]
### Masterpiece: Nuestra Obra Maestra

addDiscogs(mdbmaps, "Billboard", "8d31f7e1853f2b2a38cb37cdc92ac03d", "1813760")    ### [Lil Peanut]
### Lean Wit It, Rock Wit It

### Give Me Everything  ,  Hey Mama

addAllMusic(mdbmaps, "Billboard", "ce6a59b7abc5d76a683bf2aa188adf74", "mn0000178007")   ### [Russell Moore]
### Timeless Hits From The Past: Bluegrassed

addAllMusic(mdbmaps, "Billboard", "0b1e4a71bbd133e8050ba86a4492b04b", "mn0003744509")   ### [Ranna Royce]
### The Whoodlum Ball

addAllMusic(mdbmaps, "Billboard", "8a34e0d5cd9db90fc3f598b5f615854d", "mn0003712915")   ### [Smith And Hay]
### The Whoodlum Ball

mdbmaps["Billboard"].save()

In [ ]:
addAllMusic(mdbmaps, "Top40", "5a888aa5d5b0835533b2a73dd895b461", "mn0002528230")   ### [DCUP]
### We No Speak Americano

addAllMusic(mdbmaps, "Top40", "9d337e01e83fc2b9d9fcbd6930bfb107", "mn0003715465")   ### [Casper Magico]
### Te Bote  ,  Otro Trago

mdbmaps["Top40"].save()

In [ ]:
1/0
addAllMusic(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "mn0000417717")   ### [Raybion Bros.]
addDiscogs(mdbmaps, "Top40", "b67c4a39d846b0e14fcda8299193701a", "2188194")    ### [Raybion Bros.]
### Butterfly Kisses

addAllMusic(mdbmaps, "Top40", "00d5d59798eeffef591f272260b0bbcc", "mn0000124572")   ### [Chuck-N-Blood]
### My Dogs

addAllMusic(mdbmaps, "Top40", "866a64f5a5e684f61aa573af7f63bc57", "mn0000233423")   ### [Magoo]
### Up Jumps Da Boogie

addAllMusic(mdbmaps, "Top40", "78f1feb9204bce3173d42798bc9bcb91", "mn0001808325")   ### [Kobe Bryant]
### Hold Me

addAllMusic(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "mn0000186401")   ### [Karen O.]
addDiscogs(mdbmaps, "Top40", "d92e471232aa6a0bc4a03a49f16b0997", "245778")    ### [Karen O.]
### Hello Tomorrow

addAllMusic(mdbmaps, "Top40", "5edcd28a4505dc29e4edebd67052e614", "mn0000795894")   ### [Dolla]

addAllMusic(mdbmaps, "Top40", "7bcb752a835787affb80072d61387b22", "mn0001411030")   ### [Hollis]
### White Walls

addAllMusic(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "mn0003348925")   ### [Jennifer Lawrence]
addDiscogs(mdbmaps, "Top40", "c0051ae12f6048a807213c44b434aa1d", "4210193")    ### [Jennifer Lawrence]
### The Hanging Tree

addAllMusic(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "mn0003323773")   ### [Lookas]
addDiscogs(mdbmaps, "Top40", "083b67cc423fe5482eaf1412aebb65b3", "3619379")    ### [Lookas]
### GDFR

addAllMusic(mdbmaps, "Top40", "f37bd04269e23d0c4c10c435a4f1dd7f", "mn0002943184")   ### [Mnek]
### Never Forget You  ,  So Good

addAllMusic(mdbmaps, "Top40", "1f9b34897f6440e2484f1ac5e00953f7", "mn0003462822")   ### [Gnash]
### I Hate U, I Love U  ,  Lights Down Low

addAllMusic(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "mn0000351749")   ### [Royce Da 5'9]
addDiscogs(mdbmaps, "Top40", "f731f61bae9f9216f33f0cf76d403fee", "40792")    ### [Royce Da 5'9]
### Not Alike

addAllMusic(mdbmaps, "Top40", "296504ec4590a1522dcf6afb838fd6f4", "mn0003332851")   ### [London On Da Track]
### Numbers

mdbmaps["Top40"].save()

In [ ]:
cad.getAlbumsData("Common Ground")

In [ ]:
updateManDB(2)

# Analyze Matches

In [ ]:
toget

In [ ]:
updateManDB(2)

In [ ]:
updateManDB(4)

In [ ]:
print([item[1]["ArtistName"] for item in toMatch["Top40"]])

In [ ]:
saveData = []
for fullName in sorted(list(cad.manyArtists.keys())):
    appendName = " & ".join(["[{0}]".format(manDB.renamed(indivName)) for indivName in sorted(list(cad.manyArtists[fullName]))])
    saveData.append([fullName, {"Guess": appendName, "Truth": ''}]) # = list(cad.manyArtists[fullName].keys())
saveFile(idata=saveData, ifile="tmp.yaml")

In [ ]:
fixedData = getFile("tmp.yaml")

In [ ]:
multis = []
multirenames = {}
for item in fixedData:
    if len(item[1]["Truth"]) > 0:
        name = item[1]["Truth"]
        if len(name.split("] & [")) == 1:
            multis.append(item[1]["Truth"][1:-1])
        else:
            rename = name[1:-1].split("] & [")
            multirenames[item[0]] = " ::: ".join(rename)
        
print("Found {0} renames".format(len(multirenames)))
print("Found {0} multi-artists".format(len(multis)))

#### Save MultiArtists (if found)

In [ ]:
knownFilename = "../multiartist/knownMultiArtists.yaml"
knownMultis = getFile(knownFilename)
print(len(multis))
print(len(knownMultis))
knownMultis += multis
print(len(knownMultis))
knownMultis = sorted(list(set(knownMultis)))
print(len(knownMultis))
saveFile(idata=knownMultis, ifile=knownFilename)

#### Save multi renames (if needed)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
#multiManDB.forceReload(getFile("main.yaml"))
multiManDB.addRenames(multirenames)

In [ ]:
multiManDB = masterArtistNameDB("multi", init=False)
multiManDB.addRenames(multirenames)
multiManDB.save()
saveFile(idata=multiManDB.getRenames(), ifile="multi.yaml")
multiManDB = masterArtistNameDB("multi", init=True)
multiManDB.forceReload(getFile("multi.yaml"))
multiManDB.checkForRecursives()
multiManDB = masterArtistNameDB("multi", init=False)

In [ ]:
#### Edit main.yaml (if needed)

In [ ]:
mdbmaps["Billboard"].getDF()

In [ ]:
mdf = mdbmaps["BillboardYE"].getDF()
mdf.shape
#mdf[mdf["DBMatches"] == 0]

# Multi Match

In [ ]:
cad.getTypeArtistAlbumData("Joe Rene")

In [ ]:
toMatch["Billboard"]

In [ ]:
manualAppends(cad, mType, toMatch, chartType, minAlbums=0, add=True)

In [ ]:
mdbmc.getMasterDF("Billboard")

In [ ]:
matchItAll(mdbmaps, getThresholdsWithoutAlbums(0.85), toMatch=toMatch, useAlbums=False)
reMatch()

In [ ]:
#singleArtistAlbumData["Billboard"]['Ernest']
saveFile(idata=[x[1]["ArtistName"] for x in toMatch["Top40"]], ifile="toget.p")

# Find Near Artist Name Matches

In [ ]:
toget, dbRenames = analyzeMatches(maindb, mdbmaps, chartType, maxVal=1.0, diffVal=1.0)

In [ ]:
if len(dbRenames) > 0:
    analyzeRenames(manDB, dbRenames)
    # Test for overap with manDB

    dbRenames = getFile(ifile="relDBRenames2.yaml")
    redos = {}
    dels  = []
    for oldername,bestname in dbRenames.items():
        if oldername in manDB.artistNameDB.keys(): # and bestname not in manDB.artistNameDB.keys():
            #print("Older=[{0}] \t Best=[{1}]".format(oldername,bestname))

            redos[bestname] = oldername
            dels.append(oldername)
            #print('PROBLEM ({0} in master DB): \t{1}  -->  {2}'.format(oldername, oldername, bestname))
            try:
                matchOlderData = mdbmaps[chartType].getArtistDataByName(oldername).getDict()
            except:
                matchOlderData = {}

            try:
                matchBestData  = mdbmaps[chartType].getArtistDataByName(bestname).getDict()
            except:
                matchBestData = {}

            #print("Older: {0}".format(matchOlderData))
            #print("Best:  {0}".format(matchBestData))
            #print("\n")
        else:
            print("{0}: {1}".format(oldername, bestname))

    print("# Renames: {0}".format(len(dbRenames)))
    print("# Dels: {0}".format(len(dels)))

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()

In [ ]:
manDB = masterArtistNameDB("main", init=True)
manDB.forceReload(getFile("main.yaml"))
manDB.addRenames(getFile("relDBRenames2.yaml"))
manDB.checkForRecursives()
manDB.save()
manDB = masterArtistNameDB("main", init=False)
saveFile(idata=manDB.dbRenames, ifile="main.yaml")
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
removeDBRenames(mdbmaps, chartType, dbRenames)

In [ ]:
vals="""
[Sander Van Doom                0.9                Sander Van Doorn] 	 --> (Top40) 27813b44b4282de50171e13adb4ebcf9 / 183267 (Discogs)
"""

lines=vals.split("\n")
lines2=[x.split("({0}) ".format(chartType))[1] for x in lines if len(x) > 0]
names2=[x.split("({0}) ".format(chartType))[0] for x in lines if len(x) > 0]
lines3={x.split(" / ")[0]: x[x.find("(")+1:-1] for x in lines2}
names3=[x.split("    ")[0].strip()[1:] for x in names2]
if len(names3) != len(lines3):
    raise ValueError("Err")

for i,(k,v) in enumerate(lines3.items()):
    artistName = names3[i]
    #print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', \'{2}\')".format(k,v,))
    print("### ----> {0}".format(artistName))
    print("mdbmaps[chartType].addArtistDataByID(\'{0}\', \'{1}\', None)".format(k,v))    
    albums = singleArtistAlbumData[chartType][artistName]
    print("### {0}".format("  ,  ".join(albums)))
    print("### ignores.append(\"{0}\")".format(artistName))
    print("\n")
    
    
#mdbmaps[chartType].addArtistDataByID("e309c2e3fc905eae304b71e10e82eb99", "AllMusic", '0000803469')
print("")
print("mdbmaps[chartType].save()")
print("saveFile(idata=list(set(ignores)), ifile=\"chartIgnores.yaml\")")

In [ ]:
### ----> Sander Van Doom
mdbmaps[chartType].addArtistDataByID('27813b44b4282de50171e13adb4ebcf9', 'Discogs', None)
### Grasshopper
### ignores.append("Sander Van Doom")



mdbmaps[chartType].save()
saveFile(idata=list(set(ignores)), ifile="chartIgnores.yaml")

In [ ]:
toget

****
****
****
****

# Multis

In [ ]:
ignores.append('Daryl Hall & John Oates')

In [ ]:
from multiArtist import multiartist
mularts  = multiartist(cutoff=0.9, discdata=None, exact=False)
knownMultiArtists = getFile("../multiartist/knownMultiArtists.yaml")
print(len(knownMultiArtists))
print(len(mdbmaps[chartType].getArtists()))
for idx,artistName in mdbmaps[chartType].getArtists().items():
    N = mularts.getArtistNames(artistName)
    if len(N) > 1:
        result = {subName: mdbmaps[chartType].isKnownByName(subName) for subName in N}
        if not any(result.values()):
            knownMultiArtists.append(artistName)
            #print(result)
        #print("ignores.append(\'{0}\')".format(artistName))

print(len(knownMultiArtists))
knownMultiArtists = list(set(knownMultiArtists))
print(len(knownMultiArtists))
        
#saveFile(idata = sorted(knownMultiArtists), ifile="../multiartist/knownMultiArtists.yaml")

In [ ]:
ignores = getFile("chartIgnores.yaml")
dels = []
for idx,artistName in mdbmaps[chartType].getArtists().items():
    if artistName in ignores:
        dels.append([idx,artistName])
        
print(len(dels))
for idx,artistName in dels:
    mdbmaps[chartType].removeArtistByID(idx)
    
mdbmaps[chartType].save()

In [ ]:
len(dels)

# Functions

In [ ]:
class matchthresholds:
    def __init__(self):
        thresholds = {}
        thresholds[1000] = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 10.0}
        thresholds[500]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 5.0}
        thresholds[200]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(200/8), 'score': 2.5}
        thresholds[100]  = {'numArtistName': 1, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(100/8), 'score': 1.5}
        thresholds[50]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': int(50/8), 'score': 1.5}
        thresholds[20]   = {'numArtistName': 2, 'artistNameCutoff': 0.95, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 3, 'score': 1.5}
        thresholds[10]   = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[5]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[3]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 2, 'score': 1.5}
        thresholds[2]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 2.0}
        thresholds[1]    = {'numArtistName': 5, 'artistNameCutoff': 0.90, 'artistAlbumCutoff': 0.9, 'numArtistAlbums': 1, 'score': 0.9}
        self.thresholds = thresholds

        iterItems = {20: {"Max": 50, "Vals": 100}}
        iterItems.update({10: {"Max": 20, "Vals": 200}})
        iterItems.update({50: {"Max": 10000, "Vals": 50}})
        iterItems.update({5: {"Max": 10, "Vals": 250}})
        iterItems.update({3: {"Max": 5, "Vals": 500}})
        iterItems.update({2: {"Max": 3, "Vals": 500}})
        iterItems.update({1: {"Max": 2, "Vals": 500}})
        self.iterItems = iterItems

        self.thresholdsNoAlbums = {'numArtistName': 5, 'artistNameCutoff': 0.9, 'artistAlbumCutoff': None, 'numArtistAlbums': None, 'score': None}
    
    def getThresholdsWithAlbums(self, minAlbums):
        if self.thresholds.get(minAlbums) is not None:
            return self.thresholds[minAlbums]
        
    def getThresholdsWithoutAlbums(self, cutoff):
        self.thresholdsNoAlbums["artistNameCutoff"] = cutoff
        return self.thresholdsNoAlbums
    
    def getIterItems(self):
        return self.iterItems
        

In [ ]:
from functools import partial
from matchDBArtist import matchDBArtist

class multimatch:
    def __init__(self, maindb, mdbmc, debug=False):
        self.debug  = debug
        self.maindb = maindb
        self.mdbmc  = mdbmc
        
        self.mt = matchthresholds()
        
        self.toMatch = None
        self.dbName  = None
        self.maxMatches = None
        
        self.matchFunc = {True: self.matchDBArtistWithAlbums, False: self.matchDBArtistWithoutAlbums}
            
        
    def setMDBMap(self, dbName, mdbmap):
        self.mdbmap = mdbmap
        self.dbName = dbName
    
    
    def setDataToMatch(self, toMatch):
        if toMatch is not None:
            self.toMatch = toMatch
            
            
    def setMaxMatches(self, maxMatches):
        self.maxMatches = maxMatches
        
        
    ############################################################################################################
    ## Set Data To Be Matched
    ############################################################################################################
    def setData(self, useAlbums=True, minAlbums=None, maxValues=None, cutoff=None):
        self.useAlbums = useAlbums
        print("HI")
        if useAlbums is True:
            if minAlbums is not None:
                self.thresholds = self.mt.getThresholdsWithAlbums(minAlbums)
                iterItems = self.mt.getIterItems()[minAlbums]
                maxAlbums = iterItems["Max"]
                if maxValues is None:
                    maxValues = thresholds["Vals"]

                self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=[])
                print("Found {0} Artists To Match With Albums and Using Thresholds: {1}".format(len(self.toMatch[self.dbName]), self.thresholds))
            else:
                raise ValueError("Must supply a minAlbums value")
        else:
            self.thresholds = self.mt.getThresholdsWithoutAlbums(cutoff)
            if maxValues is None:
                maxValues = 100000
            self.toMatch   = mdbmc.getDataToMatch(self.dbName, maxValues=maxValues, maxAlbums=100000, minAlbums=0, ignores=[])
            print("Found {0} Artists To Match Without Albums and Without Using Thresholds".format(len(self.toMatch[self.dbName])))
            
    
    


    def matchItAll(self):
        num_processes = 3
        func       = self.matchFunc[self.useAlbums]
        thresholds = self.thresholds
        pfunc = partial(func, **thresholds) # Giving some arguments for kwargs
        argument_list = self.toMatch[self.dbName]
        print("Args: ",argument_list)
        
        if len(argument_list) == 0:
            return
        print("Using {0} Matching Function".format(pfunc))
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = self.multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)
        return result_list
        

    def matchDBArtistWithoutAlbums(self, item, *args, **kwargs):
        return 1

        print("Item ==>",item)
        1/0
        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = None
        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'],
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score']),
        mcs    = mdbMatcher.findPotentialArtistNameMatchesWithoutAlbums()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def matchDBArtistWithAlbums(self, item, *args, **kwargs):    
        #time.sleep(0.0025)

        return 1
        print("Item ==>",item)
        1/0
        

        primaryKey   = item[0]
        artistData   = item[1]
        artistName   = artistData["ArtistName"]
        artistID     = primaryKey
        artistAlbums = artistData["ArtistAlbums"]

        mdbMatcher = matchDBArtist(self.maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        mdbMatcher.setThresholds(matchNumArtistName=kwargs['numArtistName'], matchArtistNameCutoff=kwargs['artistNameCutoff'], 
                                 matchArtistAlbumCutoff=kwargs['artistAlbumCutoff'], matchNumArtistAlbums=kwargs['numArtistAlbums'],
                                 matchScore=kwargs['score'])
        mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
        retval = [primaryKey,artistName,artistID,mcs]
        return retval


    def multiProc(self, func, argument_list, num_processes):
        pool = Pool(processes=num_processes)
        result_list_tqdm = []
        for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
            result_list_tqdm.append(result)
        return result_list_tqdm


In [ ]:
mm = multimatch(maindb, mdbmc)
mm.setMDBMap("Billboard", mdbmaps["Billboard"])

In [ ]:
mm.setData(useAlbums=True, minAlbums=10, maxValues=1)

In [ ]:
mm.matchItAll()

In [ ]:
ignoresList = ["chartIgnores.yaml"]
for minAlbums,minAlbumsData in getIterItems().items():
    print("="*10,minAlbums)
    maxAlbums = minAlbumsData["Max"]
    maxValues = minAlbumsData["Vals"]
    for i in range(10):
        toMatch   = mdbmc.getDataToMatch(chartType, maxValues=maxValues, maxAlbums=maxAlbums, minAlbums=minAlbums, ignores=ignoresList)
        if len(toMatch[chartType]) == 0:
            break
        matchItAll(mdbmaps, getThresholds(minAlbums), toMatch=toMatch)
        
reMatch()
print("Done.\n\n")

In [ ]:
from billboardCharts import billboardCharts
from fileUtils import getBasename, getDirname, getBaseFilename
from timeUtils import clock, elapsed
from webUtils import getHTML, getWebData
from timeUtils import getDateTime, isDate
from listUtils import getFlatList
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
from os.path import join
from searchUtils import findExt
import urllib
from time import sleep
from collections import Counter
from artistIgnores import getArtistIgnores

from billboardCharts import billboardCharts
from top40Charts import top40Charts

class billboardFiles:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Billoard"

    def findFiles(self):
        savedir = join(self.basedir, "data", "billboard", "results")
        self.files   = findExt(savedir, ext='.p')
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
    
    
class billboardFullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Billboard"
        self.charts  = charts
        self.tfiles  = billboardFiles(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()
        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))


class top40files:
    def __init__(self, basedir):
        self.basedir = basedir
        self.source  = "Top40"

    def findFiles(self):
        savedir = join(self.basedir, "data", "top40")
        self.files   = findExt(savedir, ext='.p')         
        print("Found {0} {1} files.".format(len(self.files), self.source))
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        return self.files
    
    
class top40FullChart:
    def __init__(self, basedir, charts):
        self.basedir = basedir
        self.source  = "Top40"
        self.charts  = charts
        self.tfiles  = top40files(self.basedir)
        self.files   = self.tfiles.findFiles()        
        self.fullChartData = {}
        
    def filterFiles(self):
        files = {}
        for chartName, ifile in self.files.items():
            if chartName in self.charts:
                files[chartName] = ifile
        print("Keeping {0}/{1} Charts".format(len(files), len(self.files)))
        
    
        
    def setFullChartData(self):
        fullChartData = {}
        renameStats   = Counter()
        
        self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)


class chartData:
    def __init__(self, source, minYear=None, maxYear=None, country=None, debug=False):
        debug=False    
        self.source = source
        
        self.basedir  = "/Volumes/Piggy/Charts/"
        self.basename = source
        

        try:
            self.cts    = {"Billboard": billboardCharts(), "Top40": top40Charts()}[source]
            self.cFiles = {"Billboard": billboard40files(self.basedir), "Top40": top40files(self.basedir)}[source]
            self.files  = self.cFiles.findFiles()
        except:
            raise ValueError("Could not create charts data for {0}".format(source))

                        
        self.charts = []
            
        self.minYear   = minYear
        self.maxYear   = maxYear
        
        self.artistRenames   = {}
        self.dbRenames       = {}

        self.chartData       = {}
        self.fullChartData   = {}
        self.artistAlbumData = {}
        
        
    #####################################################################################################################################
    ## Artist Data
    #####################################################################################################################################
    def getArtists(self):
        return list(self.artistAlbumData.keys())
        
        
    
    #####################################################################################################################################
    ## Full Chart Data
    #####################################################################################################################################
    def getFullChartDataFilename(self):
        ifile="current{0}FullChartArtistAlbumData.p".format(self.basename)
        return ifile

    def getFullChartData(self):
        return getFile(self.getFullChartDataFilename())
        
    def saveFullChartData(self):
        print("Saving {0} Full Artist Data".format(len(self.fullChartData)))
        saveFile(idata=self.fullChartData, ifile=self.getFullChartDataFilename(), debug=True)        
        
        
    
    #####################################################################################################################################
    ## Artist Album Data
    #####################################################################################################################################
    def getArtistAlbumDataFilename(self):
        ifile="current{0}ArtistAlbumData.p".format(self.basename)
        return ifile
    
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        
    
    def getArtistAlbumData(self):
        return getFile(self.getArtistAlbumDataFilename())
        
    def saveArtistAlbumData(self):
        print("Saving {0} Artist Album Data to {1}".format(len(self.artistAlbumData), self.getArtistAlbumDataFilename()))
        saveFile(idata=self.artistAlbumData, ifile=self.getArtistAlbumDataFilename(), debug=True)  
        
        
    
    #####################################################################################################################################
    ## Rename Data
    #####################################################################################################################################        
    def setRenames(self, artistRenames):
        self.artistRenames = artistRenames
        
    def setDBRenames(self, dbRenames):
        self.dbRenames = dbRenames
        
        
    
    #####################################################################################################################################
    ## Find Files
    #####################################################################################################################################
    def setChartUsage(self, name=None, rank=None):
        if rank is not None:
            if isinstance(rank, list):
                for item in rank:
                    self.charts += self.cts.getChartsByRank(item)
            elif isinstance(rank, int):
                self.charts += self.cts.getChartsByRank(rank)
        elif name is not None:
            self.charts += self.cts.getCharts(name)
        else:
            self.charts = self.cts.getCharts(None)
        if name is None:
            name = "None"
        print("  Using Charts ({0}): {1}".format(name, self.charts))
        
        
        
                
    #####################################################################################################################################
    ## Set Artist Album Data
    #####################################################################################################################################
    def setArtistAlbumData(self):
        self.artistAlbumData = {artist: list(artistData["Songs"].keys()) + list(artistData["Albums"].keys()) for artist,artistData in self.fullChartData.items()}
        print("There are {0} unique artist entries".format(len(self.artistAlbumData)))
        

    #####################################################################################################################################
    ## Set Full Chart Data
    #####################################################################################################################################
    def setFullChartData(self):
        {"Billboard": self.setFullChartDataBillboard(), "Top40": self.setFullChartDataTop40()}[self.source]
    
    def setFullChartDataBillboard(self):        
        renameStats  = Counter()
        chartCounter = Counter()
        
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        
        for ifile in self.files:
            fdata = getFile(ifile)
            for chartName, cnameResults in fdata.items():
                if chartName not in self.charts:
                    continue
                
                for date, dResults in cnameResults.items():
                    if self.minYear is not None:
                        if getDateTime(date).year < int(self.minYear):
                            continue
                    if self.maxYear is not None:
                        if getDateTime(date).year > int(self.maxYear):
                            continue
                    stryear = getDateTime(date).year

                    artist = dResults["Artist"]

                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist 
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist  

                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue


                    chartCounter[chartName] += 1

                    album  = dResults["Name"]

                    if self.chartData.get(artist) is None:
                        self.chartData[artist] = Counter()
                    self.chartData[artist][album] += 1
                    
                    if self.fullChartData.get(artist) is None:
                        self.fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if self.fullChartData[artist][key].get(album) is None:
                        self.fullChartData[artist][key][album] = {}
                    if self.fullChartData[artist][key][album].get(chartName) is None:
                        self.fullChartData[artist][key][album][chartName] = {}
                    self.fullChartData[artist][key][album][chartName][date] = 0
                #print("{0: <40}{1}".format("{0}-{1}".format(chartName,stryear),len(self.fullChartData)))
                
        
    def setFullChartDataTop40(self):
        fullChartData = {}
        renameStats   = Counter()
        
        #self.findFiles()
        if len(self.files) == 0:
            raise ValueError("There are no files. Something is wrong...")
        self.files = {getBaseFilename(x).replace("/", " "): x for x in self.files}
        
        for chartName, ifile in self.files.items():
            if chartName not in self.charts:
                continue
            print("==> {0: <40}".format(chartName), end="\t")
            #t40chart = top40chart(chartID, chartName, chartURL)
            chartResults = getFile(ifile)

            for date, values in chartResults.items():
                if self.minYear is not None:
                    if getDateTime(date).year < int(self.minYear):
                        continue
                if self.maxYear is not None:
                    if getDateTime(date).year > int(self.maxYear):
                        continue

                        
                for i,item in enumerate(values):
                    artist = item["Artist"]
                    renamedArtist = artist
                    for testArtist in self.artistRenames.keys():
                        if artist.find(testArtist) != -1:
                            tmp = renamedArtist
                            renamedArtist = renamedArtist.replace(testArtist, self.artistRenames.get(testArtist))
                            #print("{0}  <---- From ---- {1}".format(renamedArtist, tmp))
                            renameStats[renamedArtist] += 1
                            artist = renamedArtist
                    
                    if self.dbRenames.get(artist) is not None:
                        renamedArtist = self.dbRenames[artist]
                        renameStats[renamedArtist] += 1
                        artist = renamedArtist
                    

                    artist = artist.replace("\r", "")                    
                    
                    ignoreStatus = getArtistIgnores(artist)
                    if ignoreStatus is False:
                        continue
                    
                    album  = item["Album"]
                    if album in ["Soundtrack"]:
                        continue

                    if fullChartData.get(artist) is None:
                        fullChartData[artist] = {"Songs": {}, "Albums": {}}
                    if chartName.endswith("Albums"):
                        key = "Albums"
                    else:
                        key = "Songs"
                    if fullChartData[artist][key].get(album) is None:
                        fullChartData[artist][key][album] = {}
                    if fullChartData[artist][key][album].get(chartName) is None:
                        fullChartData[artist][key][album][chartName] = {}
                    fullChartData[artist][key][album][chartName][date] = i
            print(len(fullChartData))
        self.fullChartData = fullChartData
        
        if self.artistRenames is not None:
            print("Renamed {0} artists".format(len(renameStats)))
            print("Most Common Artists:")
            for item in renameStats.most_common(5):
                print(item)